# Naked Puts Screener

### Imports

In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import datetime as dt
import math
from scipy.stats import norm
import os
import numpy as np
import pytz
import nltk
import time
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/junlo/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### Helper Functions

In [2]:
def manual_etf_tickers():
    etfs = ['SVIX', 'FLWS', 'VCXA', 'TXG', 'YQ', 'TURN', 'ATNF', 'DIBS', 'XXII', 'ME', 'TSVT', 'TWOU', 'BITX', 'UVIX', 'DDD', 'MMM', 'FDMT', 'FEAM', 'ETNB', 'EGHT', 'NMTRQ', 'MASS', 'AKA', 'AMRK', 'ATEN', 'AAON', 'AIR', 'ABT', 'ABBV', 'ABCM', 'ABCL', 'ABEO', 'ANF', 'ABM', 'BCIM', 'BCI', 'BCD', 'SGOL', 'SIVR', 'ABSI', 'ACTG', 'ASO', 'ACHC', 'ACAD', 'ACEL', 'AXDX', 'ACN', 'ACCO', 'ACCD', 'ARAY', 'ACRX', 'SLRN', 'ACER', 'ACHV', 'ACIW', 'ACRS', 'ACMR', 'ACNB', 'ACOR', 'ACR', 'ATNM', 'ATVI', 'AYI', 'ABOS', 'GOLF', 'AFIB', 'ACVA', 'PEO', 'ADMP', 'AHCO', 'ADAP', 'ADPT', 'ADCT', 'ADUS', 'AGRO', 'ADEA', 'ADIL', 'ACET', 'ADNT', 'ADTX', 'ADMA', 'ADBE', 'ADT', 'ATGE', 'ADTN', 'AAP', 'WMS', 'AEIS', 'AMD', 'ASIX', 'ADV', 'ADN', 'ADVM', 'EATV', 'GK', 'LETB', 'MSOX', 'YOLO', 'MSOS', 'HDGE', 'ACM', 'AGLE', 'AEG', 'AEHR', 'AMTX', 'AER', 'AVAV', 'ASLE', 'AES', 'AEMD', 'AEVA', 'LIDR', 'AFCG', 'AMG', 'AFMD', 'AFRM', 'AFL', 'MITT', 'AGCO', 'UAVS', 'AGEN', 'A', 'AGTI', 'AGL', 'AGYS', 'AGIO', 'AGNC', 'AEM', 'API', 'ADC', 'AGRI', 'AL', 'APD', 'ATSG', 'ABNB', 'AIRG', 'MIMO', 'AKAM', 'AKBA', 'KERN', 'AKRO', 'AKTS', 'AKYA', 'ALG', 'AGI', 'ALRM', 'ALK', 'AIN', 'ALB', 'ACI', 'AA', 'ALC', 'ALDX', 'ALEC', 'ALRS', 'ALEX', 'ARE', 'ASTL', 'AQN', 'BABA', 'ALIT', 'ALGN', 'ALHC', 'ALGS', 'ALKT', 'ALKS', 'ALLK', 'BIRD', 'ALGT', 'ALLE', 'ALLG', 'ALGM', 'ARLP', 'AB', 'LNT', 'ALNT', 'ALSN', 'ALLO', 'ALLT', 'ALVR', 'ALL', 'ALUR', 'ALLY', 'AAU', 'ALNY', 'AOSL', 'QMOM', 'AMR', 'APT', 'GOOGL', 'GOOG', 'ATEC', 'ALPP', 'ALPN', 'PINE', 'OGIG', 'OUSA', 'AMLP', 'BFOR', 'ACES', 'DTEC', 'SBIO', 'ALTG', 'ALTR', 'AYX', 'ATUS', 'ALT', 'ALTU', 'ALTO', 'MO', 'AMPS', 'ALXO', 'ALZN', 'AMAL', 'AMRN', 'AMZN', 'AMBC', 'AMBA', 'ABEV', 'AMAM', 'AMC', 'AMCX', 'AMCR', 'DOX', 'AMED', 'AMSWA', 'AWR', 'AMTB', 'AEE', 'AMRC', 'AMX', 'AAL', 'AAT', 'AXL', 'QPFF', 'AVEM', 'AVDE', 'QGRO', 'ACIC', 'AEO', 'AEP', 'AEL', 'AXP', 'AFG', 'AMH', 'AIG', 'AOUT', 'APEI', 'AREC', 'AMSC', 'AMT', 'AVD', 'AMWL', 'AMWD', 'AWK', 'CRMT', 'COLD', 'AMP', 'ABCB', 'ABC', 'ATLO', 'AME', 'AMGN', 'FOLD', 'AMKR', 'POWW', 'AMN', 'AMRX', 'AP', 'AMPH', 'APH', 'AMPY', 'BATT', 'SWAN', 'BLOK', 'DIVO', 'IWIN', 'IBUY', 'CNBS', 'AMPL', 'AMPX', 'ASYS', 'AMLX', 'ADI', 'ANAB', 'AVXL', 'ANDE', 'ANGI', 'ANGO', 'AU', 'BUD', 'ANIP', 'ANIK', 'NLY', 'ANNX', 'ANSS', 'ATEX', 'AM', 'AR', 'HOUS', 'AON', 'APA', 'AIRC', 'AIV', 'APLS', 'APG', 'APOG', 'ARI', 'APO', 'AMEH', 'APPF', 'APPHQ', 'APPN', 'APLE', 'AAPL', 'APLD', 'APDN', 'AIT', 'AMAT', 'AMTI', 'AAOI', 'APP', 'APRE', 'ATR', 'APTV', 'APTO', 'AQMS', 'AQB', 'AQST', 'ARMK', 'ARAV', 'ARBE', 'ABR', 'ABUS', 'ARC', 'RKDA', 'ARCB', 'ACLX', 'MT', 'ACGL', 'ARCH', 'ACHR', 'ADM', 'AROC', 'FUV', 'ARCE', 'ARCO', 'ACA', 'ARCT', 'RCUS', 'ARQT', 'AMBP', 'ARDX', 'ASC', 'AAC', 'ARCC', 'ACRE', 'ARES', 'ARGX', 'ARBK', 'ARGO', 'ARHS', 'ARIS', 'ANET', 'ARKQ', 'ARKF', 'ARKK', 'ARKW', 'ARKG', 'IZRL', 'ARKX', 'ARKO', 'AAIC', 'ARLO', 'AHH', 'ARR', 'AWI', 'ARQQ', 'ARRY', 'ARVL', 'ARW', 'GYLD', 'YPS', 'ARWR', 'AIP', 'APAM', 'AORT', 'ARVN', 'ASA', 'ASAN', 'ABG', 'ASND', 'ASX', 'ASXC', 'ASGN', 'AHT', 'ASH', 'ASLN', 'ASML', 'ASPN', 'ASPU', 'AZPN', 'ASMB', 'ASRT', 'AMK', 'ASB', 'AIZ', 'AGO', 'ASTS', 'ASTE', 'ASTR', 'AZN', 'ATXS', 'ATRO', 'ASUR', 'T', 'ATAI', 'ATRA', 'AVIR', 'ATER', 'ATHX', 'ATHA', 'ATI', 'ATIP', 'ATKR', 'BATRA', 'AUB', 'AY', 'ATLC', 'AESI', 'TEAM', 'ATO', 'ATMU', 'ATNI', 'ATOM', 'ATOS', 'BCEL', 'ATRC', 'LIFE', 'AUDA', 'AUDC', 'AURA', 'AUPH', 'ACB', 'AUR', 'JG', 'ADSK', 'ATHM', 'ALV', 'AUTL', 'ADP', 'AN', 'AZO', 'AVDL', 'AVB', 'AVNS', 'AVTA', 'AVTR', 'AVAH', 'AVPT', 'AVY', 'AVNW', 'CDMO', 'AVID', 'RNA', 'AVDX', 'AVNT', 'CAR', 'AVA', 'RCEL', 'AVT', 'AWRE', 'AXTA', 'ACLS', 'AXS', 'AXGN', 'AXON', 'AXNX', 'AX', 'AXSM', 'AXTI', 'AYRO', 'AZEK', 'AZTA', 'AZUL', 'AZZ', 'BGS', 'RILY', 'BTG', 'BW', 'BLZE', 'BMI', 'BIDU', 'BKR', 'BKKT', 'BCPC', 'BALL', 'BLDP', 'BALY', 'BANC', 'BBAR', 'BBVA', 'BBD', 'BCH', 'BLX', 'BMA', 'BSBR', 'BSAC', 'SAN', 'BAND', 'BAC', 'BOH', 'BMRC', 'FNGS', 'BMO', 'BK', 'BNS', 'NTB', 'OZK', 'BFIN', 'BKU', 'BANR', 'BZUN', 'BHB', 'DJP', 'GRN', 'BCS', 'BBDC', 'BARK', 'BNED', 'B', 'BBSI', 'GOLD', 'BSET', 'BBWI', 'BHC', 'BLCO', 'BAX', 'BCML', 'BTE', 'BCE', 'BECN', 'BEEM', 'BEAM', 'BZH', 'BDX', 'BGNE', 'BDC', 'BLPH', 'BRBR', 'BHE', 'BHIL', 'BSY', 'WRB', 'BERY', 'BBY', 'BEST', 'BTTX', 'BWMX', 'XAIR', 'BYND', 'BYSI', 'BGC', 'BHP', 'BCYC', 'BGFV', 'BIG', 'BBAI', 'BIGC', 'BILI', 'BILL', 'BIO', 'BKYI', 'BPTH', 'TECH', 'BCAB', 'BCDA', 'BIOC', 'BCRX', 'BDSX', 'BFRI', 'BIIB', 'BHVN', 'BLFS', 'BMRN', 'BMEA', 'BNGO', 'BNTX', 'BIOR', 'BSGM', 'BTCY', 'BVS', 'BIVI', 'BTAI', 'BRDS', 'BTBT', 'BTCM', 'BITF', 'BNMV', 'BJRI', 'BJ', 'BKH', 'BKI', 'BSM', 'BLKB', 'BB', 'BL', 'BKCC', 'LCTD', 'BLK', 'NEAR', 'TCPC', 'ICSH', 'BKSY', 'BX', 'BXMT', 'BXSL', 'BLDE', 'BLND', 'BLNK', 'HRB', 'SQ', 'BE', 'BLMN', 'APRN', 'BLBD', 'BLFY', 'OBDC', 'OWL', 'BRBS', 'BLUE', 'BVH', 'BXC', 'BPMC', 'BMTX', 'BA', 'BCC', 'BOKF', 'BKNG', 'BOOT', 'BAH', 'BWA', 'BORR', 'SAM', 'BOC', 'BXP', 'BSX', 'BOWL', 'BOX', 'BOXDQ', 'BYD', 'BP', 'BPT', 'BRC', 'BHR', 'BRAG', 'BCLI', 'BDN', 'BRZE', 'BRCC', 'BFH', 'BRFS', 'BCTX', 'BRDG', 'BBIO', 'BTWN', 'BWB', 'BGXX', 'BHG', 'BFAM', 'BCOV', 'BHF', 'BSIG', 'BRSP', 'BV', 'BRLT', 'EAT', 'BCO', 'BMY', 'VTOL', 'BTI', 'BRX', 'AVGO', 'BR', 'BNL', 'BWEN', 'BKD', 'BBU', 'BN', 'BIP', 'BIPC', 'BNRE', 'BEPC', 'BEP', 'BRKL', 'BRO', 'DOOO', 'BRKR', 'BC', 'BSQR', 'BKE', 'BFIX', 'BBW', 'BLDR', 'BMBL', 'BG', 'BUR', 'BFI', 'BURL', 'BFST', 'BFLY', 'BZFD', 'BWXT', 'BY', 'BYRN', 'CHRW', 'AI', 'CCCC', 'CABA', 'CBT', 'CACI', 'CADE', 'CDNS', 'CDZI', 'CDRE', 'CAE', 'CZR', 'CSTE', 'CALM', 'CAMP', 'CVGW', 'CALC', 'CMCL', 'CAL', 'CRC', 'CWT', 'CALX', 'CALT', 'CPE', 'CLMT', 'CEI', 'CMBM', 'TOKE', 'SYLD', 'TAIL', 'CATC', 'CAC', 'CPT', 'CCJ', 'CPB', 'CWH', 'CAMT', 'CAN', 'GOOS', 'CM', 'CNQ', 'CNI', 'CP', 'CSIQ', 'CNNE', 'CANO', 'GOEV', 'CGC', 'CTLP', 'CCBG', 'CGUS', 'CGDV', 'CGGR', 'COF', 'CPLP', 'CSWC', 'CFFN', 'TIME', 'CPRI', 'CAPR', 'CGRN', 'CARA', 'CRDF', 'CAH', 'CRDL', 'CDLX', 'CDNA', 'CMAX', 'CTRE', 'CARG', 'CRBU', 'CARM', 'CSL', 'CG', 'CGBD', 'KMX', 'CCL', 'CUK', 'PRTS', 'CRS', 'CSV', 'CARR', 'TAST', 'CARS', 'CARE', 'CRI', 'CVNA', 'CASA', 'CWST', 'CASY', 'CASI', 'SAVA', 'CSTL', 'CTRM', 'CTLT', 'CPRX', 'CAT', 'CATY', 'CATO', 'CAVA', 'CVCO', 'CZOO', 'CBAT', 'YCBD', 'CBZ', 'CBOE', 'VIX', 'CBRE', 'CCCS', 'CDW', 'CECO', 'FUN', 'CVM', 'CE', 'CELC', 'CLS', 'CLDX', 'CLBT', 'CLLS', 'CELH', 'CELU', 'CX', 'CENN', 'CVE', 'CNC', 'CNP', 'CSR', 'CNTA', 'CENTA', 'CPF', 'CEPU', 'CVCY', 'LEU', 'CENX', 'CNTY', 'CCS', 'IPSC', 'CRNT', 'CRNC', 'CERE', 'CDAY', 'CERT', 'CERS', 'CEVA', 'CF', 'GIB', 'CSBR', 'CHX', 'CRGE', 'CHPT', 'CRL', 'GTLS', 'CHTR', 'CCF', 'CLDT', 'CHKP', 'CKPT', 'CAKE', 'CMCM', 'CHEF', 'CHGG', 'CHE', 'CC', 'LNG', 'CQP', 'CHK', 'CVX', 'CHWY', 'CSSE', 'CHS', 'PLCE', 'CIM', 'CMRX', 'CAAS', 'CYD', 'CD', 'CMG', 'CHH', 'CHRD', 'CDXC', 'CB', 'CHT', 'CHD', 'CCV', 'CVII', 'CHDN', 'CHUY', 'CINT', 'CIG', 'CDTX', 'CIEN', 'CMPR', 'CINF', 'CNK', 'CTAS', 'CION', 'CIFR', 'CIR', 'CRUS', 'CSCO', 'CISO', 'CTRN', 'C', 'CTXR', 'CFG', 'CIA', 'CIO', 'CVEO', 'CIVI', 'CLVT', 'CMTG', 'CLAR', 'CLNE', 'CLH', 'CLSK', 'CCO', 'YOU', 'CLFD', 'CLPT', 'CLSD', 'CLIR', 'CWAN', 'CLW', 'CWEN', 'CLNN', 'CLF', 'CLVR', 'CLPR', 'RAASY', 'CLX', 'NET', 'CLOV', 'CME', 'CMS', 'CNA', 'CNHI', 'CNO', 'CNX', 'CODX', 'KO', 'CCEP', 'KOF', 'CDXS', 'CDE', 'CCOI', 'CGNX', 'CTSH', 'CGNT', 'CNS', 'COHR', 'CHRS', 'COHU', 'COIN', 'CL', 'SPCX', 'COLL', 'CIGI', 'CLGN', 'COLB', 'ECON', 'CLBK', 'COLM', 'CMCO', 'CMCSA', 'CMA', 'FIX', 'CBSH', 'CMC', 'CVGI', 'COMM', 'CBU', 'CYH', 'CHCT', 'CTBI', 'CVLT', 'CBD', 'SBS', 'ELP', 'SID', 'CCU', 'BVN', 'CODI', 'COMP', 'CMP', 'CMPS', 'CGEN', 'CPSI', 'CTG', 'SCOR', 'LODE', 'CRK', 'CMTL', 'CAG', 'CNXC', 'CNDT', 'CFLT', 'CNMD', 'CNTB', 'CNOB', 'CONN', 'COP', 'CCSI', 'CEIX', 'CNSL', 'ED', 'CWCO', 'STZ', 'CEG', 'CSTM', 'CPSS', 'TCS', 'WISH', 'VLRS', 'CONX', 'COO', 'CPS', 'CPA', 'CPRT', 'CORT', 'CNM', 'CLB', 'CRBG', 'CXW', 'CORR', 'CRMD', 'GLW', 'CAAP', 'OFC', 'CRSR', 'CTVA', 'CRVS', 'COSM', 'CMRE', 'CSGP', 'COST', 'CTRA', 'COTY', 'BASE', 'CPNG', 'COUR', 'CUZ', 'CVLG', 'CPSH', 'CRAI', 'CBRL', 'CR', 'CXT', 'BAP', 'CACC', 'CRDO', 'CCAP', 'CPG', 'CEQP', 'CRESY', 'CRH', 'CRCT', 'CRNX', 'CRSP', 'CRTO', 'CROX', 'CRON', 'CCRN', 'CAPL', 'CFB', 'CRWD', 'CCI', 'CRWS', 'CCK', 'CYRX', 'LAW', 'CSGS', 'CCLP', 'CSWI', 'CSX', 'CTO', 'CTS', 'CUBE', 'CUE', 'HLTH', 'CFR', 'CGEM', 'CULP', 'CPIX', 'CMI', 'CMLS', 'CVAC', 'CURI', 'CRIS', 'CURO', 'CW', 'CWK', 'CTOS', 'CUBI', 'CUTR', 'CVBF', 'CVI', 'UAN', 'CVRX', 'CVS', 'CYBR', 'CYCN', 'CBAY', 'CYN', 'CTKB', 'CYTK', 'CTSO', 'DHI', 'DADA', 'DAKT', 'DAN', 'DHR', 'DAC', 'DNMR', 'DQ', 'DRI', 'DRIO', 'DAR', 'DSKE', 'DAIO', 'DDOG', 'PLAY', 'DTEAF', 'DVA', 'DAWN', 'DBVT', 'NAIL', 'ASHS', 'ASHR', 'CN', 'DBEF', 'DBEU', 'DBJP', 'HYLB', 'SNPE', 'DBTX', 'DCPH', 'DECK', 'DE', 'DH', 'DCTH', 'DKL', 'DK', 'DELL', 'DAL', 'DLX', 'DNLI', 'DEN', 'DNN', 'DENN', 'XRAY', 'DMTK', 'DSGX', 'DSGN', 'DBI', 'DM', 'DESP', 'DXLG', 'DB', 'DVN', 'DXCM', 'DHX', 'DHT', 'DEO', 'DMAC', 'DO', 'FANG', 'DRH', 'DSX', 'DKS', 'DGII', 'DMRC', 'DGLY', 'DLR', 'APPS', 'DWAC', 'DBRG', 'DOCN', 'DCOM', 'DFIP', 'DFIV', 'DFAU', 'DFUS', 'DUHP', 'DFUV', 'DFAR', 'DFSV', 'DFAX', 'DFIC', 'DIN', 'DIOD', 'TMV', 'TMF', 'TYO', 'TYD', 'COM', 'BRZU', 'CHAU', 'CWEB', 'WEBL', 'ERY', 'FAZ', 'YANG', 'DUST', 'JNUG', 'EDZ', 'INDL', 'DRN', 'DRV', 'DPST', 'UBOT', 'HIBS', 'LABD', 'HIBL', 'TZA', 'TECS', 'TPOR', 'YINN', 'AAPD', 'AAPU', 'DFEN', 'WANT', 'EDC', 'ERX', 'FAS', 'EURL', 'NUGT', 'DUSL', 'JDST', 'MIDU', 'MEXX', 'DRIP', 'SPUU', 'SPDN', 'SPXS', 'SOXL', 'SOXS', 'TNA', 'TECL', 'TSLS', 'UTSL', 'SPXL', 'CURE', 'HJEN', 'MOON', 'QQQE', 'GUSH', 'RETL', 'LABU', 'TSLL', 'IRON', 'DFS', 'DISH', 'DIS', 'DHC', 'DLHC', 'DLO', 'BOOM', 'DCBO', 'DCGO', 'DOCU', 'DOGZ', 'DLB', 'DOLE', 'DG', 'DLTR', 'DLPN', 'DOMA', 'D', 'DPZ', 'DOMO', 'DCI', 'DFIN', 'DASH', 'DMLP', 'LPG', 'DORM', 'DSL', 'DV', 'PLOW', 'DOUG', 'DEI', 'DOYU', 'DOV', 'DOW', 'DJX', 'DOCS', 'RDY', 'DKNG', 'DPRO', 'DRD', 'DFH', 'DRQ', 'DSHK', 'DRVN', 'DBX', 'DTM', 'DTE', 'NAPA', 'DCO', 'DUK', 'DNB', 'DUOL', 'DD', 'DRRX', 'BROS', 'DXC', 'DXPE', 'DYAI', 'DY', 'DLNG', 'DT', 'DVAX', 'DYN', 'DX', 'DZSI', 'ELF', 'ETWO', 'BSVO', 'DRLL', 'EGBN', 'EGLE', 'EXP', 'EGRX', 'ESTE', 'EWBC', 'DEA', 'EBC', 'EMN', 'KODK', 'ETN', 'EBON', 'EBAY', 'EBET', 'EBIX', 'SATS', 'ECL', 'EC', 'ECVT', 'EDAP', 'EPC', 'EGIO', 'EIX', 'EDIT', 'EW', 'EFTR', 'EGAN', 'EH', 'EHTH', 'EIGR', 'EKSO', 'LOCO', 'ELAN', 'ESTC', 'EGO', 'SOLO', 'EA', 'ELDN', 'ESI', 'ELV', 'LLY', 'EFC', 'EARN', 'ELME', 'ELYS', 'EMAN', 'EMBC', 'ERJ', 'EME', 'EMKR', 'EEX', 'SOL', 'EBS', 'EMR', 'ESRT', 'EMX', 'ACT', 'ENTA', 'ENB', 'EHC', 'ECPG', 'WIRE', 'DAVA', 'EDR', 'EXK', 'ENR', 'WATT', 'UUUU', 'ERII', 'SIXE', 'ET', 'NRGV', 'EPAC', 'ERF', 'ENS', 'NETI', 'ENFN', 'ESMT', 'NETZ', 'ENG', 'EHAB', 'E', 'ENLT', 'ENLC', 'ELVN', 'ENVA', 'ENOV', 'ENVX', 'ENPH', 'NPO', 'ENSV', 'ENSG', 'ESGR', 'ENTG', 'ENTX', 'ETR', 'EBTC', 'EFSC', 'EPD', 'EGLX', 'EVC', 'ELA', 'ENV', 'NVRI', 'EVTV', 'NVST', 'EVA', 'NVNO', 'ENZ', 'EOG', 'EOSE', 'EPAM', 'PLUS', 'EPR', 'EPSN', 'EQOSQ', 'EQRX', 'EQT', 'EFX', 'EQ', 'EQIX', 'EQNR', 'EQX', 'EQH', 'ETRN', 'EQBK', 'EQC', 'EQR', 'ERAS', 'ERIC', 'ERIE', 'ZGN', 'ERO', 'ESAB', 'ESE', 'ESPR', 'GWH', 'EPIX', 'ESNT', 'EPRT', 'WTRG', 'ESS', 'ESTA', 'ERNA', 'BDRY', 'ITEQ', 'MJ', 'AWAY', 'HACK', 'SILJ', 'GERM', 'IVES', 'HDRO', 'GFOF', 'RIET', 'NUSI', 'DEEP', 'SEA', 'JETS', 'PFFA', 'ETD', 'ETSY', 'EURN', 'EEFT', 'EWCZ', 'EVAX', 'EVEX', 'EVLO', 'EB', 'EVBG', 'EVCM', 'EVR', 'EG', 'EVRG', 'EVRI', 'EVER', 'ES', 'MRAM', 'EVTC', 'EVGO', 'EVGN', 'EVH', 'EOLS', 'EPM', 'EVLV', 'EXAS', 'EE', 'TDSC', 'NVQ', 'QRFT', 'BITQ', 'KOIN', 'EMQQ', 'ROBO', 'XELA', 'EXEL', 'EXC', 'EXLS', 'EXPI', 'EXPE', 'EXPD', 'EXFY', 'EXPO', 'EXPR', 'XPRO', 'EXAI', 'EXR', 'EXTR', 'XOM', 'EYEN', 'EYPT', 'EZPW', 'FNB', 'FG', 'FFIV', 'FN', 'FDS', 'FICO', 'FANH', 'FFIE', 'FTCH', 'FMAO', 'FMNB', 'FPI', 'FARO', 'FAST', 'FSLY', 'FATE', 'FATH', 'FTHM', 'FAZE', 'TTAC', 'AGM', 'FRT', 'FSS', 'FHI', 'FDX', 'FENC', 'FERG', 'RACE', 'GSM', 'FGH', 'FGEN', 'FRNW', 'ONEQ', 'FSTA', 'FDVV', 'FIDI', 'FDMO', 'FCOM', 'FDIS', 'FENY', 'FNCL', 'FHLC', 'FIDU', 'FTEC', 'FMAT', 'FREL', 'FUTY', 'FQAL', 'FSMD', 'FBND', 'FNF', 'FIS', 'FDUS', 'FRGI', 'FITB', 'FIGS', 'FOA', 'FISI', 'SIXM', 'FNCH', 'FNGR', 'FINV', 'FA', 'FAF', 'FNLC', 'FBNC', 'FBP', 'FBMS', 'FRBA', 'FCBC', 'FCF', 'FCNCA', 'FFBC', 'THFF', 'FFWM', 'FHB', 'FHN', 'FR', 'INBK', 'FIBK', 'AG', 'FMBH', 'FRCB', 'FSLR', 'FPE', 'FCA', 'FDT', 'FEMS', 'FEM', 'FEUZ', 'FEP', 'FGM', 'FJP', 'FSZ', 'FTGC', 'SKYY', 'FXD', 'FXG', 'FDNI', 'FXN', 'FXH', 'FXR', 'FTAG', 'FXZ', 'CIBR', 'FXL', 'FXU', 'FPX', 'FTCS', 'FDN', 'FAN', 'FTSM', 'LMBS', 'QCLN', 'FCG', 'FBT', 'FV', 'TDIV', 'ROBT', 'SDVY', 'CRPT', 'FIXD', 'FEX', 'FDL', 'QQEW', 'QTEC', 'QABA', 'FRI', 'FVD', 'FTC', 'FWRG', 'FCFS', 'FE', 'SVVC', 'FSV', 'NOTE', 'FI', 'FSR', 'FIVE', 'FIVN', 'FVRR', 'FLT', 'FLNG', 'FLEX', 'IQDF', 'GUNR', 'QDEF', 'FND', 'FTK', 'FLO', 'FLS', 'FLNC', 'FLNT', 'FLR', 'FLUX', 'FLYW', 'FMC', 'FOCS', 'FCUV', 'FHTX', 'FMX', 'FL', 'F', 'FRSX', 'FOR', 'FRGE', 'FORM', 'FORR', 'FBRX', 'FTNT', 'FTS', 'FTV', 'FTRE', 'FBIO', 'FSM', 'FBIN', 'FET', 'FWRD', 'FOSL', 'FSTR', 'FOXA', 'FOX', 'FOXF', 'FNV', 'FBRT', 'FELE', 'HELX', 'BEN', 'FSP', 'FLJP', 'FLQL', 'FRHC', 'FCX', 'RAIL', 'FREQ', 'FMS', 'FDP', 'FRPT', 'FRSH', 'FREY', 'FTDR', 'FYBR', 'ULCC', 'FRO', 'FSK', 'HUGE', 'IGLD', 'RDVI', 'FTAI', 'FTCI', 'FCN', 'FUBO', 'FTEK', 'FCEL', 'FULC', 'FLGT', 'FLL', 'YMM', 'FUL', 'FULT', 'FNKO', 'HTOO', 'FUSN', 'FUTU', 'FTFT', 'FF', 'FVCB', 'GIII', 'GTHX', 'GAIA', 'GANX', 'GLPG', 'GALT', 'GLTO', 'GRTX', 'GAU', 'AJG', 'GAMB', 'GAME', 'GME', 'GMDA', 'GLPI', 'GAN', 'GCI', 'GOTU', 'GPS', 'GRMN', 'GTX', 'IT', 'GTES', 'GATO', 'GATX', 'GCMG', 'GDS', 'GEHC', 'GEN', 'GNSS', 'GNK', 'GENC', 'WGS', 'GNRC', 'GD', 'GE', 'GIS', 'GM', 'GBIO', 'GCO', 'GEL', 'GENE', 'GNE', 'GNS', 'GENI', 'G', 'GNPX', 'GNTX', 'THRM', 'GPC', 'GNW', 'GEO', 'GPRK', 'GEOS', 'GGB', 'GERN', 'GETY', 'GTY', 'GEVO', 'GFL', 'ROCK', 'GILT', 'GIL', 'GILD', 'DNA', 'GTLB', 'GBCI', 'GLAD', 'GOOD', 'GAIN', 'LAND', 'GLT', 'GKOS', 'VRAR', 'GB', 'GBTG', 'GLBE', 'GIC', 'GMRE', 'GLP', 'GPN', 'GSL', 'GWRS', 'AGNG', 'AIQ', 'DRIV', 'POTX', 'AQWA', 'CTEC', 'CLOU', 'KRMA', 'BUG', 'DJIA', 'EBIZ', 'FYLG', 'FINX', 'ARGT', 'ASEA', 'HYDR', 'GNOM', 'GURU', 'MLPX', 'MLPA', 'GXG', 'KROP', 'COPX', 'GOEX', 'SIL', 'URA', 'DIV', 'HYLG', 'TYLG', 'LIT', 'MILN', 'CHIQ', 'CHIC', 'CHIE', 'CHIX', 'CHII', 'CHIK', 'CHIM', 'CHIR', 'GREK', 'EMFM', 'SDEM', 'QYLD', 'QRMI', 'QYLG', 'PFFV', 'BOTZ', 'RYLD', 'RYLG', 'CATH', 'XYLD', 'XYLG', 'XRMI', 'SOCL', 'SDIV', 'EDOC', 'GXTG', 'PFFD', 'HERO', 'WNDY', 'EMBD', 'PAVE', 'GFS', 'GSAT', 'GLOB', 'GL', 'GLBS', 'GMED', 'GLYC', 'GMS', 'GDDY', 'GOGO', 'GGR', 'GOCO', 'GOL', 'GLNG', 'GFI', 'GORO', 'GROY', 'GDEN', 'GMGI', 'GOGL', 'GLOV', 'GSBD', 'GSEW', 'GBIL', 'GHYB', 'GIGB', 'GCOR', 'GEM', 'GSIE', 'GSLC', 'GDEF', 'GVIP', 'GINN', 'JUST', 'GS', 'GBDC', 'GDRX', 'GT', 'GSHD', 'GPRO', 'GRC', 'GOSS', 'GRAB', 'GGG', 'EAF', 'GHM', 'GWW', 'GTE', 'LOPE', 'GVA', 'GPMT', 'TSL', 'CONL', 'AAPB', 'COMB', 'GPK', 'GRPH', 'GTN', 'AJX', 'GLDD', 'GSBC', 'GRBK', 'GDOT', 'GPRE', 'GPP', 'GBX', 'GHL', 'GREE', 'GLRE', 'GP', 'GEF', 'GDYN', 'GFF', 'GRFS', 'GRND', 'GRTS', 'GO', 'GPI', 'GRPN', 'GROV', 'GRWG', 'OMAB', 'ASR', 'GGAL', 'SUPV', 'TV', 'GSIT', 'GSK', 'GTBP', 'GNTY', 'GH', 'GES', 'GWRE', 'GIFI', 'GPOR', 'GXO', 'HEES', 'HTHT', 'HCKT', 'HAE', 'HGTY', 'HAIN', 'HOFV', 'HNRG', 'HAL', 'HALO', 'HLNE', 'HWC', 'HBI', 'HAFC', 'HASI', 'HONE', 'HOG', 'HLIT', 'HRMY', 'HMY', 'HROW', 'HCOM', 'HIG', 'HBIO', 'HAS', 'HCP', 'HVT', 'HE', 'HA', 'HWKN', 'HAYN', 'HAYW', 'HBT', 'HCA', 'HCI', 'HDB', 'SIXV', 'HCAT', 'HR', 'HCSG', 'HQY', 'PEAK', 'HSTM', 'HTLD', 'HTLF', 'HL', 'HEI', 'HSII', 'HELE', 'HLGN', 'HLIO', 'HLX', 'MOMO', 'HP', 'JKHY', 'HSIC', 'HEPA', 'HLF', 'HRI', 'HTGC', 'HCCI', 'HFWA', 'HRTG', 'HRTX', 'HT', 'HSY', 'HTZ', 'HES', 'HESM', 'HPE', 'HXL', 'HFFG', 'DINO', 'HIBB', 'HITI', 'SNLN', 'HPK', 'HIW', 'HI', 'HLMN', 'HTH', 'HGV', 'HLT', 'HIMX', 'HIMS', 'HIPO', 'HRT', 'HIVE', 'HNI', 'HLLY', 'HEP', 'HOLI', 'HOLX', 'HBCP', 'HOMB', 'HD', 'HMST', 'FIXX', 'HMC', 'HNST', 'HON', 'HOFT', 'HOOK', 'HRZN', 'HZNP', 'HRL', 'HST', 'TWNK', 'HUSA', 'HHH', 'HWM', 'HPQ', 'HSBC', 'HUBG', 'HUBS', 'HBM', 'HPP', 'HDSN', 'HUMA', 'HUM', 'JBHT', 'HBAN', 'HII', 'HUN', 'HURN', 'HUT', 'HCM', 'HUYA', 'H', 'HYMC', 'HYFM', 'HYLN', 'HYPR', 'HYREQ', 'HY', 'HYZN', 'IMAB', 'IIIV', 'IAC', 'IAG', 'IBEX', 'IBIO', 'ICAD', 'IEP', 'ICHR', 'IBN', 'ICL', 'ICLK', 'ICLR', 'ICVX', 'ICUI', 'IDA', 'IDR', 'IDEX', 'IDYA', 'INVE', 'IEX', 'IDXX', 'IDT', 'IESC', 'IGC', 'IGMS', 'IHRT', 'IHS', 'ITW', 'ILMN', 'IMAX', 'IMBIQ', 'IMTX', 'IMMR', 'IMUX', 'IBRX', 'IMCR', 'IMGN', 'IMNM', 'IPA', 'IMVT', 'IMMP', 'IMO', 'IMPP', 'PI', 'NARI', 'INCY', 'ICD', 'IRT', 'INDB', 'IBCP', 'QAI', 'MNA', 'INDI', 'ILPT', 'SIXI', 'INFN', 'INFI', 'INFA', 'INFY', 'INFU', 'ING', 'IR', 'IMKTA', 'INGR', 'INBX', 'INMD', 'INMB', 'INOD', 'IOSP', 'INNV', 'VATE', 'IIPR', 'FFTY', 'LOUP', 'PSEP', 'INVA', 'INVZ', 'INGN', 'NOTV', 'INO', 'INZY', 'INSG', 'NSIT', 'INSM', 'NSP', 'ISPO', 'INSP', 'INSE', 'NSPR', 'IBP', 'IIIN', 'TIL', 'INST', 'PODD', 'INTA', 'ITGR', 'IART', 'IAS', 'INTC', 'NTLA', 'IDN', 'IPAR', 'IBKR', 'ICPT', 'ICE', 'INCR', 'IDCC', 'TILE', 'IBOC', 'IBM', 'IFF', 'IGT', 'IP', 'INSW', 'IPG', 'INTT', 'IVAC', 'ITCI', 'IPI', 'INTZ', 'INTU', 'ISRG', 'IVT', 'SJIM', 'GSY', 'EVMT', 'PDBC', 'SATO', 'FXA', 'FXB', 'FXC', 'FXE', 'FXY', 'FXF', 'DBC', 'DBA', 'DBB', 'DBE', 'DBO', 'DBP', 'UDN', 'UUP', 'PIZ', 'PIE', 'PDP', 'DWAS', 'PXE', 'PWZ', 'PCEF', 'PCY', 'PHB', 'PXH', 'PBD', 'KBWB', 'KBWD', 'KBWP', 'KBWY', 'CUT', 'QQQJ', 'PGX', 'SPVU', 'SPHB', 'RWL', 'EELV', 'CGW', 'RWK', 'XMLV', 'RWJ', 'PSCE', 'PSCH', 'SPHD', 'SPLV', 'TAN', 'BKLN', 'VRP', 'PPA', 'PKW', 'PGF', 'PRF', 'PRFZ', 'PSP', 'PGJ', 'PEY', 'PID', 'PNQI', 'SPGP', 'XLG', 'XMMO', 'XMHQ', 'XMVM', 'RFV', 'XSVM', 'RZV', 'CSD', 'RSPD', 'RSPG', 'RSPF', 'RSPH', 'RSPN', 'RSPM', 'RSPR', 'RSPT', 'RSP', 'RPV', 'SPHQ', 'PHO', 'PBW', 'CVY', 'PTF', 'PIN', 'PEJ', 'IVZ', 'IVR', 'PXJ', 'PJP', 'QQQ', 'PSI', 'NVDS', 'TARK', 'THCX', 'SARK', 'TSLQ', 'NVTA', 'INVH', 'IVVD', 'IONS', 'IONQ', 'IOVA', 'VXZ', 'VXX', 'IPGP', 'IQ', 'IQV', 'IRTC', 'IRDM', 'IREN', 'IRBT', 'IRM', 'IRNT', 'IRWD', 'IRS', 'USHY', 'ICF', 'ICVT', 'IAGG', 'IEFA', 'HEFA', 'IGV', 'IAUF', 'IAU', 'IEMG', 'DVYE', 'ESGE', 'FM', 'FILL', 'LEMB', 'VEGI', 'EWA', 'EWK', 'BKF', 'EWZ', 'EWC', 'EWQ', 'RING', 'EWG', 'EWH', 'EWI', 'SCJ', 'EWJ', 'EWW', 'EWM', 'EWN', 'EPP', 'EWS', 'EWP', 'EZA', 'EWY', 'EWD', 'EWL', 'EWT', 'THD', 'TUR', 'GVI', 'HYXU', 'IDV', 'CEMB', 'REM', 'ECH', 'CNYA', 'EFG', 'EFAV', 'EFV', 'EEMV', 'EZU', 'PICK', 'ACWV', 'SLVP', 'INDA', 'USMV', 'MTUM', 'QUAL', 'VLUE', 'IGE', 'GSG', 'SLV', 'STIP', 'SHYG', 'SHY', 'IGLB', 'TLH', 'TLT', 'IEI', 'IEF', 'AIA', 'FXI', 'ECNS', 'ISTB', 'DGRO', 'HDV', 'IEUR', 'IPAC', 'IXUS', 'IJH', 'IJR', 'ITOT', 'IUSG', 'IUSV', 'IVV', 'IUSB', 'AGG', 'USRT', 'HEZU', 'HEWG', 'IHAK', 'SCZ', 'SUSB', 'IEV', 'IGM', 'XT', 'FALN', 'IXP', 'IDNA', 'ICLN', 'WOOD', 'IGF', 'IXJ', 'IOO', 'IXC', 'IXG', 'MXI', 'REET', 'IXN', 'HEWJ', 'HYG', 'LQD', 'IFGL', 'WPS', 'INTF', 'IGOV', 'IBB', 'SOXX', 'IGSB', 'IGIB', 'EMB', 'ILF', 'MBB', 'IWC', 'ILCG', 'IMCG', 'AAXJ', 'ACWI', 'ACWX', 'MCHI', 'EFA', 'EEM', 'EUFN', 'IQLT', 'IVLU', 'EWU', 'SUSA', 'SIZE', 'IGN', 'MUB', 'ENZL', 'PFF', 'REZ', 'IRBO', 'IWB', 'IWF', 'IWD', 'IWO', 'IWN', 'IWP', 'IWS', 'IWR', 'IWX', 'IWM', 'IWV', 'OEF', 'IVW', 'IVE', 'IJK', 'IJJ', 'IJT', 'DVY', 'IDRV', 'SHV', 'IJS', 'TIP', 'TFLO', 'LRGF', 'IYM', 'IYE', 'IYF', 'IHI', 'IHE', 'IYR', 'IYW', 'IDU', 'IAI', 'IYC', 'IYH', 'IHF', 'IEZ', 'IAT', 'SMLF', 'USIG', 'IWFH', 'CMDY', 'CCRV', 'COMT', 'LQDH', 'HYGH', 'ITA', 'IEDI', 'ITB', 'IFRA', 'IEO', 'IYZ', 'IYT', 'GOVT', 'SVAL', 'ISUN', 'ITP', 'ITUB', 'ITOS', 'ITRI', 'ITT', 'IZEA', 'JPEM', 'JPIN', 'JEPI', 'JIRE', 'JPUS', 'JEPQ', 'JPST', 'JQUA', 'JVAL', 'JBL', 'JACK', 'JXN', 'J', 'JAGX', 'JHX', 'JRVR', 'JAMF', 'JMBS', 'JAAA', 'JHG', 'JBI', 'JANX', 'JSPR', 'JAZZ', 'JBGS', 'JD', 'JEF', 'JELD', 'JBLU', 'FROG', 'JKS', 'JOAN', 'JOBY', 'JBT', 'JHML', 'JMSB', 'JNJ', 'JCI', 'JYNT', 'JLL', 'YY', 'BBCA', 'BBEU', 'AMJ', 'JPM', 'JCPI', 'JMST', 'JMIA', 'JNPR', 'KAI', 'KALU', 'KLTR', 'KALV', 'KAMN', 'KNDI', 'BZ', 'TOON', 'KRTX', 'KPTI', 'KPLT', 'KYN', 'KB', 'KBH', 'KBR', 'BEKE', 'KRNY', 'K', 'KELYA', 'KMT', 'KW', 'KEN', 'KVUE', 'KROS', 'KDP', 'KTCC', 'KEY', 'KEYS', 'KZR', 'KFRC', 'KVSA', 'KRC', 'KE', 'KRP', 'KMB', 'KIM', 'KMI', 'KNTK', 'KC', 'KNSA', 'KNTE', 'KGC', 'KNSL', 'KEX', 'KIRK', 'KRG', 'KKR', 'KREF', 'KLAC', 'KLXE', 'KNF', 'KNX', 'KNOP', 'KN', 'KOD', 'KSS', 'PHG', 'KTB', 'KOPN', 'KOP', 'KEP', 'KFY', 'KRNT', 'KRMD', 'KOS', 'KHC', 'KCCA', 'KWEB', 'KARS', 'KMED', 'KEUA', 'KRBN', 'KTEC', 'BNDD', 'IVOL', 'KTOS', 'DNUT', 'KR', 'KRON', 'KRO', 'KRYS', 'KT', 'KBNT', 'KLIC', 'KULR', 'KURA', 'KYMR', 'KD', 'LHX', 'LZB', 'LH', 'LADR', 'LKFN', 'LAKE', 'LRCX', 'LAMR', 'LW', 'LE', 'LSEA', 'LSTR', 'LNTH', 'LNZA', 'LGO', 'LRMR', 'LVS', 'LTCH', 'SWIM', 'LSCC', 'RODM', 'EL', 'LAUR', 'LAZ', 'LCII', 'LFLY', 'LPTX', 'LEA', 'LZ', 'LEGN', 'LEG', 'LDOS', 'LMAT', 'LMND', 'LC', 'TREE', 'LEN', 'LII', 'DRS', 'LSAK', 'LESL', 'LEVI', 'LXRX', 'LX', 'LPL', 'LGIH', 'LI', 'LICY', 'LBRDA', 'LBRDK', 'LBRT', 'LBTYA', 'LBTYK', 'LILAK', 'LSXMA', 'FWONK', 'FWONA', 'LSXMK', 'LTRPA', 'LTH', 'LFCR', 'LFMD', 'LFST', 'LFVN', 'LGND', 'LNW', 'LTBR', 'ZEV', 'LSPD', 'LWLG', 'LILM', 'LMB', 'LMNR', 'LINC', 'LECO', 'LNC', 'LIN', 'LNN', 'LCTX', 'LGHL', 'LPCN', 'YVRLF', 'LQDA', 'LQDT', 'BAD', 'CCOR', 'INFL', 'OVLH', 'OVT', 'METV', 'MEME', 'BETZ', 'NERD', 'SPRX', 'TILL', 'LAD', 'LAC', 'DBMF', 'LFUS', 'LIVN', 'LYV', 'LOB', 'LTHM', 'LVO', 'LPSN', 'RAMP', 'LVOX', 'LVWR', 'LIZI', 'LKQ', 'LL', 'LYG', 'LMFA', 'LDI', 'LOCL', 'LMT', 'L', 'LGMK', 'LOGI', 'LOMA', 'LOOP', 'RIDEQ', 'LTRY', 'LPX', 'LOVE', 'LOW', 'LPLA', 'LXU', 'LYTS', 'LCID', 'LU', 'LULU', 'LVLU', 'LUMN', 'LITE', 'LAZR', 'LUNA', 'LXFR', 'LXP', 'LYEL', 'LYFT', 'LYB', 'MTB', 'MDC', 'MHO', 'MAC', 'MTSI', 'MGNX', 'M', 'SHOO', 'MSGE', 'MSGS', 'MDGL', 'MAG', 'MMP', 'MGTA', 'MGIC', 'MGA', 'MX', 'MGNI', 'MGY', 'MHLD', 'SECT', 'MAIN', 'TMAT', 'MMYT', 'MBUU', 'TUSK', 'TPYP', 'MANU', 'MANH', 'MNTX', 'MTW', 'MNKD', 'MAN', 'MFC', 'MARA', 'MRO', 'MPC', 'MRVI', 'MMI', 'MCS', 'MRIN', 'MPX', 'HZO', 'MRNS', 'MKL', 'MKTX', 'MKFG', 'MQ', 'MAR', 'VAC', 'MMC', 'MRTN', 'MLM', 'MMLP', 'MRVL', 'MAS', 'MASI', 'DOOR', 'MTZ', 'MBC', 'MA', 'MTDR', 'MTCH', 'MTLS', 'SIXB', 'MTRN', 'MATV', 'MTRX', 'MATX', 'MAT', 'MTTR', 'MATW', 'MXCT', 'MAXN', 'MMS', 'MXL', 'MEC', 'MBI', 'MKC', 'MCD', 'MUX', 'MCK', 'MDU', 'MFIN', 'MDVL', 'MAX', 'MPW', 'MNOV', 'MED', 'MDWD', 'MEDP', 'MDT', 'MEIP', 'MGTX', 'MLCO', 'MELI', 'MBWM', 'MERC', 'MBIN', 'MRK', 'MCY', 'MRCY', 'MLNK', 'MMSI', 'MTH', 'MACK', 'MRSN', 'MRUS', 'MESA', 'MSB', 'MESO', 'MMAT', 'META', 'MTA', 'MEOH', 'MEI', 'MET', 'MCBS', 'MCB', 'MTD', 'MFA', 'MGEE', 'MTG', 'MGM', 'MGPI', 'MCHP', 'MU', 'MSFT', 'MSTR', 'MVST', 'MVIS', 'MPB', 'MAA', 'MFIC', 'MIDD', 'MSBI', 'MOFG', 'MIST', 'MLR', 'MLKN', 'TIGO', 'MDXG', 'MNDO', 'MNMD', 'MIND', 'MTX', 'MRUT', 'XSP', 'MNSO', 'MRTX', 'MIR', 'MIRO', 'MIRM', 'AVO', 'MCW', 'MG', 'MITK', 'MUFG', 'MIXT', 'MFG', 'MKSI', 'MBLY', 'MODN', 'MRNA', 'MOD', 'MODV', 'MC', 'MOGO', 'MHK', 'MTEM', 'MOH', 'TAP', 'MNTS', 'MCRI', 'MNDY', 'MDLZ', 'ML', 'MDB', 'MPWR', 'MNRO', 'MRCC', 'MNST', 'MNTK', 'MEG', 'MCO', 'MLTX', 'CAF', 'MS', 'CVIE', 'CVLC', 'CDEI', 'CVSE', 'MORN', 'MORF', 'MOS', 'MPAA', 'MSI', 'MOV', 'MOXC', 'MP', 'MPLX', 'COOP', 'MRC', 'MSA', 'MSM', 'MXEA', 'MXEF', 'MSCI', 'MLI', 'MWA', 'MULN', 'MPLN', 'MUR', 'MUSA', 'MVO', 'MYE', 'MYMD', 'MYRG', 'MYGN', 'MYTE', 'NABL', 'NAAS', 'NBR', 'NNDM', 'NNOX', 'NANOS', 'NSTG', 'NNVC', 'NSSC', 'NDX', 'NDAQ', 'NTRA', 'FIZZ', 'NBHC', 'NCMI', 'NFG', 'NGG', 'NHC', 'NATI', 'NRC', 'NSA', 'EYE', 'NTCO', 'NGS', 'NGVC', 'NHTC', 'NRP', 'NATR', 'NWG', 'NAUT', 'NLS', 'NAVB', 'NAVI', 'NVGS', 'NM', 'NMM', 'NVTS', 'NBTB', 'NCNO', 'NCR', 'NKTR', 'NNI', 'NMRD', 'NGMS', 'NEOG', 'NEO', 'NLTX', 'NEON', 'NRDS', 'NRDY', 'NTAP', 'NTES', 'NFLX', 'NTGR', 'NTCT', 'NTWK', 'NTST', 'NBIX', 'STIM', 'NMTC', 'NVRO', 'NFE', 'NFGC', 'NGD', 'NMFC', 'EDU', 'NEWP', 'NEWR', 'NYCB', 'NYT', 'NEGG', 'NWL', 'NMRK', 'NEU', 'NEM', 'NR', 'NWSA', 'NWS', 'NEWT', 'NEXA', 'NXE', 'NXDT', 'NXRT', 'NXST', 'NEXT', 'KIND', 'NEE', 'NEP', 'NXGN', 'NEX', 'NXT', 'NGL', 'NICE', 'NHWK', 'NKE', 'NKLA', 'NINE', 'NIO', 'NI', 'NIU', 'NKTX', 'LASR', 'NMIH', 'NNBR', 'NNN', 'NOAH', 'NE', 'NOK', 'NOMD', 'NMR', 'NDLS', 'NAT', 'NDSN', 'JWN', 'NSC', 'NOA', 'NECB', 'NAK', 'GDVD', 'NOG', 'NSTB', 'NTRS', 'NFBK', 'NOC', 'NWN', 'NWPX', 'NWE', 'NCLH', 'NOV', 'NVMI', 'NG', 'NOVT', 'NVS', 'NVAX', 'NVO', 'NVCR', 'DNOW', 'NRG', 'NRXP', 'NU', 'NUS', 'BURU', 'NUE', 'NRIX', 'SMR', 'NS', 'NTNX', 'NTR', 'NUVL', 'NUVA', 'NUVB', 'NAN', 'NVEI', 'NVVE', 'NVEE', 'NVT', 'NVDA', 'NXPI', 'NYMXF', 'OI', 'OCSL', 'OTLY', 'OBE', 'OXY', 'OCEA', 'OPTT', 'OII', 'OCFC', 'OCGN', 'OCUL', 'OCUP', 'OCN', 'ODD', 'OMEX', 'OPAD', 'OPI', 'OFG', 'OFS', 'OGE', 'OIS', 'OKTA', 'OLPX', 'ODFL', 'ONB', 'ORI', 'OSBC', 'OLMA', 'OLN', 'OLK', 'OLLI', 'OLO', 'ZEUS', 'OHI', 'OMGA', 'OMER', 'OMCL', 'OMC', 'ONON', 'ON', 'ONTF', 'OCX', 'ONTX', 'ONCT', 'ONDS', 'OGS', 'OLP', 'OSS', 'OMF', 'OKE', 'OSPN', 'OSW', 'ONEW', 'ONTO', 'OTRK', 'OOMA', 'OPBK', 'OPAL', 'LPRO', 'OTEX', 'OPEN', 'KAR', 'OPRA', 'OPK', 'OPRT', 'OPY', 'OPFI', 'OPRX', 'OPCH', 'ORCL', 'ORMP', 'ORAN', 'OSUR', 'OIG', 'ORC', 'ORLY', 'OGI', 'ORGO', 'OGN', 'ONVO', 'ORGS', 'ORIC', 'OCG', 'OBK', 'ORGN', 'OESX', 'ORN', 'ONL', 'OEC', 'ORLA', 'ORA', 'OFIX', 'KIDS', 'OSCR', 'OSK', 'OSIS', 'OR', 'OTIS', 'OTMO', 'OTTR', 'OUST', 'OB', 'OUT', 'OM', 'OSG', 'OSTK', 'OVID', 'OVV', 'OMI', 'OC', 'OWLT', 'OXM', 'OXSQ', 'PX', 'PCAR', 'ODDS', 'AFTY', 'SRVR', 'INDS', 'ALTL', 'PALC', 'PTNQ', 'PTBD', 'PTLC', 'COWZ', 'CALF', 'PACB', 'PCRX', 'PKG', 'PTVE', 'PACW', 'PGY', 'PD', 'PAGS', 'PLTR', 'PTN', 'PANW', 'PLMR', 'PAM', 'PAAS', 'PANL', 'PZZA', 'PARR', 'PAR', 'FNA', 'PARAA', 'PARA', 'PGRE', 'PRTK', 'PKE', 'PCYG', 'PK', 'PRK', 'PKOH', 'PH', 'PSN', 'PASG', 'CASH', 'PAX', 'PATK', 'PDCO', 'PTEN', 'PAVM', 'PAYX', 'PAYC', 'PYCR', 'PCTY', 'PAY', 'PAYO', 'PYPL', 'PSFE', 'PAYS', 'PBF', 'CNXN', 'PCB', 'PDD', 'PDFS', 'PDSB', 'BTU', 'PKST', 'PSO', 'PEB', 'MD', 'PEGA', 'PTON', 'PBA', 'PENN', 'PNTG', 'PFLT', 'PNNT', 'PRET', 'PFSI', 'PMT', 'PAG', 'PNR', 'PEN', 'PEBO', 'PEP', 'PRSO', 'PRDO', 'PWP', 'PRFT', 'PFGC', 'PFMT', 'PRM', 'PERI', 'PPIH', 'PBT', 'PR', 'PVL', 'PRGO', 'WOOF', 'PETQ', 'PETS', 'PBR', 'PFE', 'PFSW', 'PCG', 'PGTI', 'PMCB', 'PHAT', 'PHXM', 'CELL', 'PM', 'PSX', 'PECO', 'FENG', 'PLAB', 'PHR', 'PHUN', 'PHX', 'DOC', 'PLL', 'PDM', 'PIRS', 'PPC', 'HYS', 'LTPZ', 'ZROZ', 'BOND', 'MINT', 'MUNI', 'PNFP', 'PNW', 'PINS', 'PXD', 'PIPR', 'PBI', 'PXLW', 'PJT', 'PAA', 'PAGP', 'PLNT', 'PL', 'PLG', 'PLYA', 'AGS', 'MYPS', 'PLTK', 'PLBY', 'PLXS', 'PLRX', 'PLUG', 'PLUR', 'PLYM', 'PMVP', 'PNC', 'PNM', 'MAGA', 'PII', 'PSNY', 'POL', 'PYPD', 'POOL', 'BPOP', 'PRCH', 'PTLO', 'POR', 'PTMN', 'PKX', 'POST', 'PSTL', 'PBPB', 'PCH', 'POWL', 'POWI', 'PWFL', 'PWSC', 'PPG', 'PPL', 'PRAA', 'PRAX', 'PGEN', 'PRPO', 'DTIL', 'PDS', 'PRLD', 'PFC', 'PINC', 'PBH', 'TROW', 'PSMT', 'PRME', 'PRI', 'FRST', 'PRMW', 'PRIM', 'YLD', 'USMC', 'PFG', 'PRVA', 'PRA', 'PRCT', 'PCOR', 'PG', 'UFO', 'PROF', 'ACDC', 'PRG', 'PRGS', 'PGR', 'PGNY', 'PROK', 'PLD', 'PGRU', 'PUMP', 'PRPH', 'PRQR', 'PRO', 'OILK', 'SMDV', 'TMDV', 'NOBL', 'TDV', 'REGL', 'SVXY', 'BITO', 'ZSL', 'KOLD', 'SCO', 'UCO', 'BOIL', 'UGL', 'AGQ', 'YCL', 'YCS', 'EUO', 'GLL', 'VERS', 'EET', 'QQQA', 'OND', 'ONLN', 'MYY', 'EFZ', 'EUM', 'DOG', 'DXD', 'PST', 'RXL', 'BIB', 'SSO', 'SAA', 'USD', 'DDM', 'QLD', 'ROM', 'UPW', 'UWM', 'TBT', 'BITI', 'PSQ', 'SH', 'TBF', 'SJB', 'REK', 'RWM', 'SBB', 'YXI', 'EEV', 'SPXN', 'SPXT', 'SUPL', 'EZJ', 'TTT', 'SSG', 'REW', 'SDP', 'XPP', 'BZQ', 'URE', 'SMDD', 'SRTY', 'UST', 'SKYU', 'URTY', 'UBT', 'LTL', 'UCYB', 'DIG', 'UYG', 'UYM', 'TQQQ', 'SPXU', 'SQQQ', 'SZK', 'DUG', 'SIJ', 'SMN', 'SDOW', 'SDS', 'UDOW', 'UPRO', 'QID', 'SKF', 'FXP', 'EPV', 'RXD', 'MZZ', 'EWV', 'SRS', 'TWM', 'SDD', 'BIS', 'UVXY', 'VIXM', 'VIXY', 'PSEC', 'PRSR', 'PB', 'PTGX', 'PLX', 'TARA', 'PRTA', 'PRLB', 'PVBC', 'PFS', 'PRU', 'PSQH', 'PTC', 'PTCT', 'PSA', 'PEG', 'PUBM', 'LUNG', 'PLSE', 'PHM', 'PBYI', 'PSTG', 'PCT', 'PPBT', 'PRPL', 'PVH', 'PYR', 'QTWO', 'QGEN', 'QFIN', 'QRVO', 'QUAD', 'KWR', 'QCOM', 'QLYS', 'NX', 'PWR', 'QTRX', 'QUBT', 'QSI', 'QS', 'QD', 'DGX', 'QRHC', 'QUIK', 'QDEL', 'QNCX', 'QNST', 'QIPT', 'QUOT', 'QRTEA', 'QTTOY', 'RCM', 'RXT', 'RDN', 'RLGT', 'RADI', 'RDNT', 'RDWR', 'RFL', 'RAIN', 'RL', 'METC', 'RMBS', 'RRC', 'RANI', 'PACK', 'RPD', 'RAPT', 'FLTN', 'RAVE', 'RJF', 'RYAM', 'RYN', 'RBA', 'UTWY', 'UTHY', 'XBIL', 'RBC', 'RICK', 'RCMT', 'RMAX', 'RDI', 'RC', 'SIXRE', 'O', 'RETA', 'RCON', 'RXRX', 'RCAT', 'RRGB', 'RRR', 'RDFN', 'RDW', 'RWT', 'REE', 'REED', 'RRX', 'REG', 'REGN', 'RPHS', 'RGNX', 'RM', 'RF', 'RGLS', 'RGA', 'REKR', 'RLAY', 'RS', 'RLMD', 'RELX', 'MARK', 'RELY', 'IPOS', 'IPO', 'RNR', 'RNLX', 'RNST', 'RNW', 'RENB', 'RENT', 'RTO', 'RPTX', 'RPAY', 'RGEN', 'REPL', 'FRBK', 'RSG', 'REFR', 'RSVR', 'HAUS', 'REZI', 'RMD', 'RGP', 'QSR', 'ROIC', 'RVP', 'REVG', 'RVNC', 'RVPH', 'RVMD', 'RVLV', 'RVTY', 'RWLK', 'REX', 'REXR', 'REYN', 'RH', 'RBBN', 'RELL', 'RIGL', 'RGTI', 'REPX', 'RMNI', 'REI', 'RNG', 'RIO', 'RIOT', 'RSKD', 'RAD', 'RITM', 'RVSB', 'RIVN', 'RLI', 'RLJ', 'RLX', 'RMR', 'RHI', 'HOOD', 'RBLX', 'RKT', 'RKLB', 'RCKT', 'ROK', 'RMTI', 'RCI', 'ROG', 'ROIV', 'ROKU', 'ROL', 'ROP', 'ROST', 'WEED', 'ROVR', 'RY', 'RCL', 'RGLD', 'RPRX', 'RES', 'RPM', 'RPT', 'RTX', 'RBT', 'RUM', 'RMBL', 'RWAY', 'RUSHA', 'RSI', 'RUI', 'RLV', 'RUT', 'RVLP', 'RXO', 'RXST', 'RYAN', 'RYAAY', 'R', 'RYI', 'RHP', 'RVYL', 'OEX', 'XEO', 'SPESG', 'SPGI', 'SANW', 'SBRA', 'SABR', 'SACH', 'SGBX', 'SB', 'SFE', 'SAFE', 'SAFT', 'SAGE', 'SAIA', 'CRM', 'SBH', 'IOT', 'SJT', 'SANA', 'SD', 'SAND', 'JBSS', 'SGMO', 'SANM', 'SNY', 'SAP', 'SPNS', 'SAR', 'SRPT', 'SSL', 'SATL', 'BFS', 'SVRA', 'SBAC', 'SCSC', 'SLB', 'SNDR', 'SCHN', 'SRRK', 'SCHL', 'SDGR', 'SCHW', 'SCHK', 'SCHE', 'SCHR', 'SCHF', 'SCHC', 'FNDX', 'FNDA', 'FNDE', 'FNDF', 'SCHO', 'SCHZ', 'SCHB', 'SCHD', 'SCHG', 'SCHV', 'SCHX', 'SCHM', 'SCHH', 'SCHA', 'SCHP', 'SAIC', 'SCLX', 'SCPL', 'STNG', 'SMG', 'SCPH', 'SSP', 'SRL', 'SCU', 'SCYX', 'SE', 'SA', 'SEAC', 'SBCF', 'SMHI', 'SDRL', 'STX', 'SGEN', 'SEE', 'SHIP', 'SEAS', 'SCWX', 'SEEL', 'SEER', 'SEIC', 'SEM', 'XLC', 'XLE', 'XLF', 'XLI', 'XLRE', 'XLY', 'XLP', 'XLV', 'XLU', 'XLB', 'XLK', 'WTTR', 'SELB', 'SIGI', 'SLQT', 'SLS', 'SRE', 'SEMR', 'SMTC', 'ASAI', 'ST', 'SENS', 'SXT', 'SRTS', 'S', 'MCRB', 'SRG', 'SCI', 'SVC', 'NOW', 'SFBS', 'SES', 'SEVN', 'SFL', 'SHAK', 'SHPW', 'SHCR', 'SHEL', 'SHEN', 'SHW', 'SFT', 'FOUR', 'PIXY', 'SHG', 'SHLS', 'SWAV', 'SCVL', 'SHOP', 'SSTK', 'SHYF', 'SIBN', 'SBSW', 'SIEB', 'SIEN', 'BSRR', 'SIFY', 'SIGA', 'SGHT', 'SASI', 'SGML', 'SIG', 'SLGN', 'SLAB', 'SIMO', 'SILK', 'SBOW', 'SVM', 'SILV', 'SMWB', 'SFNC', 'SPG', 'TUA', 'PFIX', 'SVOL', 'VCAR', 'SMPL', 'SSD', 'SLP', 'SBGI', 'OMIC', 'SGLY', 'SIOX', 'BLCN', 'SIRI', 'SPNT', 'SITC', 'SITE', 'SITM', 'STR', 'SIX', 'TSLX', 'SKM', 'SKX', 'SKE', 'SKIL', 'SKLZ', 'SKY', 'SKYT', 'SKYW', 'SWKS', 'SLG', 'SNBR', 'SLM', 'SLRC', 'SM', 'SGH', 'SND', 'SMBK', 'SMAR', 'SDC', 'SNN', 'SWBI', 'AOS', 'SMSI', 'SJM', 'SNAP', 'SNA', 'SNPO', 'SNDL', 'SNOW', 'SY', 'SLAC', 'SQM', 'SCTL', 'SOFI', 'SHCO', 'SOHU', 'SLGL', 'SEDG', 'SOI', 'SWI', 'SLDP', 'DTC', 'SLNH', 'SLGC', 'SOND', 'SAH', 'SEVCQ', 'SON', 'SONO', 'SONY', 'SOPH', 'SRNEQ', 'SHC', 'SOUN', 'SSTI', 'SPFI', 'SO', 'SCCO', 'SFST', 'SBSI', 'SSB', 'LUV', 'SWN', 'SOVO', 'SP', 'LOV', 'SPTN', 'DIA', 'GLD', 'GMF', 'RWO', 'RWX', 'FEZ', 'GNR', 'CWI', 'EEMX', 'QWLD', 'SPDW', 'SPEM', 'GXC', 'EDIV', 'EWX', 'DWX', 'GWX', 'NANR', 'SPY', 'MDY', 'XAR', 'CWB', 'TIPX', 'BIL', 'BILS', 'EBND', 'JNK', 'IBND', 'BWX', 'FLRN', 'SJNK', 'XSW', 'RWR', 'WIP', 'XHE', 'PSK', 'LQIG', 'NZUS', 'SHM', 'HYMB', 'XES', 'SPMB', 'SPTI', 'SPIP', 'SPTM', 'SPAB', 'SPBO', 'SPIB', 'SPLB', 'SPTL', 'SPMD', 'SPLG', 'SPSM', 'SPTS', 'SPSB', 'SPYG', 'SPYD', 'SPYV', 'ONEO', 'MDYG', 'MDYV', 'SLYG', 'SLYV', 'XBI', 'KBE', 'KCE', 'SDY', 'XHB', 'KIE', 'KOMP', 'XME', 'XOP', 'XPH', 'KRE', 'XRT', 'XSD', 'XTN', 'MMTM', 'SPB', 'ANY', 'SPHR', 'SPI', 'SPIR', 'SR', 'SPR', 'SAVE', 'SRC', 'SBEV', 'SPLK', 'SPOK', 'SRAD', 'SPWH', 'SPOT', 'SWTX', 'CXM', 'URNJ', 'URNM', 'SII', 'SPT', 'SFM', 'SPRU', 'SPSC', 'SPX', 'SPXC', 'SQSP', 'SRAX', 'SSNC', 'SRLN', 'GAL', 'INKM', 'RLY', 'TOTL', 'SSRM', 'JOE', 'STAA', 'STAG', 'STGW', 'LAB', 'SLI', 'SMP', 'SXI', 'SWK', 'STN', 'SBLK', 'SGU', 'STHO', 'SBUX', 'SRT', 'STWD', 'STT', 'STKH', 'GASS', 'STLD', 'SCS', 'STLA', 'SCM', 'STEM', 'SCL', 'STEP', 'SRCL', 'STE', 'STER', 'STRL', 'STVN', 'STC', 'SF', 'SFIX', 'STM', 'SYBT', 'STOK', 'STNE', 'SRI', 'SNEX', 'SSYS', 'STRA', 'GLDB', 'NZRO', 'HNDL', 'LRN', 'SDIG', 'SYK', 'RGR', 'SPH', 'SMFG', 'SMMF', 'INN', 'SUM', 'SMLP', 'SMMT', 'SUI', 'SNCY', 'SLF', 'SXC', 'SU', 'SUNL', 'NOVA', 'SUN', 'STKL', 'SPWR', 'RUN', 'SHO', 'SUNW', 'SGHC', 'SLGG', 'SMCI', 'SUP', 'SUPN', 'SURF', 'SURG', 'SGRY', 'SRDX', 'SSSS', 'STRO', 'SG', 'SLVM', 'SYM', 'SYNA', 'SNCR', 'SYF', 'SNDX', 'SYNH', 'SNPS', 'SNV', 'SYPR', 'SYRS', 'SYY', 'SST', 'TMUS', 'TTOO', 'TBLA', 'TRHC', 'TSM', 'TTWO', 'TAK', 'TKAT', 'TAL', 'TLIS', 'TALK', 'TALO', 'TNDM', 'SKT', 'TPR', 'TRGP', 'TGT', 'TH', 'TARS', 'TGB', 'TASK', 'TMHC', 'TRP', 'SNX', 'TDCX', 'TEL', 'FTI', 'TTGT', 'TECK', 'TGLS', 'TK', 'TNK', 'TGNA', 'TRC', 'TELA', 'TDOC', 'TEO', 'TDY', 'TFX', 'VIV', 'TEF', 'TDS', 'TSAT', 'TBIO', 'TELL', 'TLS', 'TU', 'TIXT', 'TMPO', 'TPX', 'TENB', 'TS', 'TNYA', 'TME', 'THC', 'TNC', 'TDC', 'TER', 'WULF', 'TEX', 'TX', 'TERN', 'LLAP', 'TRNO', 'TBNK', 'TSLA', 'TTEK', 'TTI', 'TAGS', 'CORN', 'SOYB', 'CANE', 'WEAT', 'TEVA', 'TCBI', 'TXN', 'TXRH', 'TGH', 'TXT', 'TFII', 'TFSL', 'TGTX', 'THCH', 'PRNT', 'AAN', 'AREN', 'BODY', 'SKIN', 'CI', 'LEV', 'RTL', 'ODP', 'STKS', 'REAX', 'REAL', 'TTD', 'NCTY', 'TXMD', 'THTX', 'TBPH', 'TMO', 'TRI', 'THO', 'TWKS', 'TDUP', 'THRY', 'PP', 'OARK', 'NVDY', 'CPII', 'AHOY', 'RPAR', 'HFND', 'UPAR', 'CHAT', 'TDW', 'TTSH', 'TLYS', 'TLRY', 'TIMB', 'TKR', 'TMST', 'TIO', 'TIPT', 'TWI', 'TITN', 'TJX', 'TMC', 'TOST', 'TOL', 'TR', 'BLD', 'MODG', 'TRMD', 'TTC', 'TD', 'CURV', 'TTE', 'TSEM', 'TSQ', 'TM', 'TPG', 'TRTX', 'TPIC', 'TSCO', 'TW', 'COOK', 'TT', 'TACT', 'TAC', 'TDG', 'TMDX', 'RIG', 'TRU', 'TNL', 'TRV', 'TVTX', 'TG', 'THS', 'TRMR', 'TRVN', 'TREX', 'TPH', 'TCBK', 'TCN', 'TMQ', 'TRS', 'TRMB', 'TNET', 'TRIB', 'TRIN', 'TRN', 'TSE', 'TCOM', 'TRIP', 'TPVG', 'DCFC', 'TRTN', 'TFIN', 'TGI', 'TRVG', 'TROX', 'TROO', 'TBI', 'TRUE', 'TFC', 'TRUP', 'SPC', 'TRST', 'TRMK', 'TRX', 'TNP', 'TTEC', 'TTMI', 'TCX', 'TOUR', 'TUP', 'TKC', 'TPB', 'HEAR', 'TSP', 'TPC', 'TUYA', 'TWLO', 'TWIN', 'TWST', 'TWO', 'TYL', 'TYRA', 'TSN', 'UHAL', 'UCAR', 'GROW', 'USAU', 'USPH', 'SLCA', 'UBER', 'UI', 'UBS', 'UDMY', 'UDR', 'UFPI', 'UFPT', 'UGI', 'PATH', 'ULTA', 'UCTT', 'RARE', 'UGP', 'UMBF', 'UMH', 'UAA', 'UA', 'UFI', 'OACP', 'UL', 'UNP', 'QURE', 'UIS', 'UAL', 'UFCS', 'UMC', 'UNFI', 'UPS', 'URI', 'UAMY', 'USM', 'X', 'UNL', 'USL', 'BNO', 'CPER', 'USCI', 'UGA', 'UNG', 'USO', 'UTHR', 'UNH', 'UNIT', 'UNTY', 'U', 'UVV', 'OLED', 'UEIC', 'UHT', 'UHS', 'UVE', 'UTI', 'UVSP', 'UNM', 'TIGR', 'UPBD', 'UPXI', 'UPH', 'UPLD', 'UPST', 'UPWK', 'UEC', 'UROY', 'UE', 'UONE', 'URBN', 'URGN', 'USB', 'USFD', 'USAC', 'USNA', 'SIXU', 'UTZ', 'UWMC', 'UXIN', 'VFC', 'VVX', 'EGY', 'VCSA', 'VCNX', 'MTN', 'VAL', 'VALE', 'VLN', 'VLO', 'WGMI', 'BTF', 'VLY', 'VMI', 'VVV', 'VNDA', 'XBTF', 'MOO', 'BIZD', 'BBH', 'BRF', 'GLCN', 'CNXT', 'DAPP', 'EINC', 'ANGL', 'BJK', 'GDX', 'RAAX', 'IHY', 'ISRA', 'EMLC', 'GDXJ', 'SMOG', 'MORT', 'CRAK', 'OIH', 'PPH', 'PFXF', 'REMX', 'RTH', 'SMH', 'BUZZ', 'SLX', 'ESPO', 'HYD', 'MLN', 'OUNZ', 'SMOT', 'MOAT', 'SMB', 'VOOG', 'VOOV', 'BIV', 'BLV', 'BSV', 'BND', 'BNDX', 'VXF', 'VUG', 'VV', 'VOT', 'VOE', 'VO', 'VNQ', 'VOO', 'VBR', 'VB', 'VBK', 'VTI', 'VTV', 'VEU', 'VWO', 'VGK', 'VPL', 'VSS', 'VNQI', 'VT', 'VTIP', 'VTEB', 'VCIT', 'VGIT', 'VCLT', 'VGLT', 'VMBS', 'VGSH', 'VCSH', 'VTC', 'VONE', 'VTWG', 'VTWO', 'VIG', 'VXUS', 'VEA', 'VFMO', 'VFMF', 'VFQY', 'VWOB', 'VYM', 'VIGI', 'VYMI', 'EDV', 'MGC', 'MGV', 'MGK', 'VOX', 'VCR', 'VDC', 'VDE', 'VFH', 'VHT', 'VIS', 'VGT', 'VAW', 'VPU', 'VAPO', 'VREX', 'VRNS', 'VBLT', 'VXRT', 'PCVX', 'VBIV', 'VGR', 'VECO', 'VEEV', 'VLD', 'VEL', 'VNTRQ', 'VTR', 'VRA', 'VERA', 'VCYT', 'MDRX', 'VSTM', 'VCEL', 'VRNT', 'VRE', 'VRSN', 'VRSK', 'VRTV', 'VERI', 'VZ', 'VET', 'VRNA', 'VRRM', 'VRCA', 'VTNR', 'VRTX', 'EVTL', 'VRT', 'VERU', 'VERV', 'VIA', 'VVI', 'DSP', 'VSAT', 'VTRS', 'VIAV', 'RBOT', 'VICI', 'VICR', 'VSCO', 'VCTR', 'CFO', 'CFA', 'CDC', 'CSB', 'VSDA', 'VIEW', 'VRAYQ', 'VKTX', 'VFF', 'VMEO', 'VWE', 'VNOM', 'VIPS', 'VQS', 'VIR', 'VIRX', 'SPCE', 'VRDN', 'VHC', 'VTSI', 'VIRT', 'V', 'VSH', 'VPG', 'VISL', 'VIST', 'VGZ', 'VSTO', 'VTGN', 'VC', 'VST', 'COCO', 'VTLE', 'VITL', 'VANI', 'SEAT', 'VVPR', 'VVOS', 'VZIO', 'VNET', 'VOC', 'VOD', 'VNRX', 'VNT', 'VOR', 'VNO', 'VOXX', 'VOYA', 'VYGR', 'VRM', 'VTEX', 'VMC', 'VUZI', 'WTI', 'WNC', 'WAB', 'WBA', 'WD', 'WKME', 'WBX', 'WMT', 'WRBY', 'WBD', 'WMG', 'HCC', 'WAFD', 'WASH', 'WCN', 'WM', 'WAT', 'WSBF', 'WSO', 'WTS', 'WVE', 'W', 'WDFC', 'WFRD', 'WEAV', 'WBS', 'WEC', 'WB', 'WEJOQ', 'WFC', 'WELL', 'WEN', 'HOWL', 'WERN', 'WCC', 'WFG', 'WST', 'WABC', 'WAL', 'WMC', 'WDC', 'WES', 'WU', 'WLKP', 'WLK', 'WPRT', 'WRK', 'WEST', 'WWR', 'WE', 'WEX', 'WY', 'WPM', 'UP', 'WHR', 'WHF', 'WSR', 'FREE', 'WOW', 'WLY', 'WLDN', 'WMB', 'WSM', 'WTW', 'WSC', 'WIMI', 'WING', 'WGO', 'WTFC', 'WIT', 'WISA', 'WBAT', 'GDE', 'WT', 'USDU', 'CYB', 'WCLD', 'XSOE', 'ELD', 'DEM', 'CEW', 'DGS', 'GCC', 'HEDJ', 'DFE', 'USFR', 'DNL', 'EPI', 'AIVI', 'DLS', 'IHDG', 'DXJ', 'DFJ', 'DXJS', 'AIVL', 'DLN', 'DON', 'DGRW', 'EES', 'AGGY', 'USMF', 'WKEY', 'WIX', 'MAPS', 'WNS', 'WOLF', 'WWW', 'WDS', 'WWD', 'WDAY', 'WKHS', 'WK', 'WKSP', 'WRLD', 'WKC', 'WWE', 'WOR', 'WPC', 'WPP', 'WRAP', 'WSFS', 'WW', 'WH', 'WYNN', 'XFOR', 'XBIT', 'XEL', 'XNCR', 'XBIO', 'XHR', 'XENE', 'XERS', 'XRX', 'XOMA', 'XMTR', 'XOS', 'XP', 'XPEL', 'XPEV', 'XPO', 'XPOF', 'XNET', 'XYL', 'YMAB', 'YALA', 'YSG', 'YELP', 'YETI', 'YEXT', 'YRD', 'YORW', 'DAO', 'YPF', 'YUM', 'YUMC', 'ZLAB', 'ZBRA', 'ZDGE', 'ZEPP', 'ZETA', 'ZVRA', 'ZH', 'ZD', 'ZG', 'Z', 'ZIM', 'ZBH', 'ZIMV', 'ZION', 'ZIP', 'ZTS', 'ZM', 'ZI', 'ZS', 'ZTO', 'ZUMZ', 'ZUO', 'ZWS', 'ZYME', 'ZYNE', 'ZYXI', 'UVIX', 'MMM', 'ABT', 'ABBV', 'ANF', 'ASO', 'ACN', 'ATVI', 'ADBE', 'AAP', 'AMD', 'MSOS', 'AFRM', 'AFL', 'AGNC', 'ABNB', 'ALB', 'AA', 'BABA', 'ALGN', 'ALLY', 'APT', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMBA', 'AMC', 'AAL', 'AEO', 'AXP', 'AIG', 'AMSC', 'AMGN', 'ADI', 'AVXL', 'BUD', 'NLY', 'AR', 'APA', 'APO', 'AAPL', 'APLD', 'AMAT', 'AAOI', 'MT', 'ADM', 'ANET', 'ARKK', 'ARKG', 'ASAN', 'ASML', 'ASTS', 'AZN', 'T', 'TEAM', 'ACB', 'ADSK', 'ADP', 'CAR', 'AXSM', 'BIDU', 'BAC', 'OZK', 'GOLD', 'BBWI', 'BHC', 'BAX', 'BBY', 'BYND', 'BBAI', 'BILI', 'BILL', 'BIIB', 'BB', 'BLK', 'BX', 'BLNK', 'SQ', 'BA', 'BKNG', 'BSX', 'BP', 'BBIO', 'BMY', 'AVGO', 'BURL', 'AI', 'CZR', 'CCJ', 'CPB', 'CWH', 'GOOS', 'CSIQ', 'CANO', 'GOEV', 'CGC', 'COF', 'CAH', 'KMX', 'CCL', 'CVNA', 'SAVA', 'CAT', 'VIX', 'CELH', 'CNC', 'CF', 'CHPT', 'CHTR', 'CC', 'LNG', 'CVX', 'CHWY', 'CMG', 'CSCO', 'C', 'CLF', 'CLX', 'NET', 'CLOV', 'CME', 'CODX', 'KO', 'COHR', 'COIN', 'CMCSA', 'CMA', 'CAG', 'COP', 'STZ', 'CSTM', 'WISH', 'GLW', 'COST', 'CTRA', 'COTY', 'CPNG', 'CRSP', 'CROX', 'CRWD', 'CSX', 'CVS', 'CBOE', 'DHI', 'DHR', 'DDOG', 'ASHR', 'DE', 'DELL', 'DAL', 'DB', 'DVN', 'FANG', 'DKS', 'DLR', 'APPS', 'DWAC', 'TMF', 'FAZ', 'DUST', 'JNUG', 'DPST', 'LABD', 'TZA', 'YINN', 'ERX', 'FAS', 'NUGT', 'JDST', 'SPXS', 'SOXL', 'SOXS', 'TNA', 'SPXL', 'LABU', 'TSLL', 'DFS', 'DISH', 'DIS', 'DOCU', 'DG', 'DLTR', 'DPZ', 'DASH', 'DOW', 'DKNG', 'DBX', 'DD', 'BROS', 'DT', 'DJX', 'EBAY', 'EBIX', 'EDIT', 'EA', 'ELV', 'LLY', 'EMR', 'ET', 'ENVX', 'ENPH', 'EPD', 'EOG', 'EOSE', 'EQT', 'JETS', 'ETSY', 'EXPE', 'EXPR', 'XOM', 'FFIE', 'FTCH', 'FSLY', 'FDX', 'RACE', 'FGEN', 'FITB', 'FHN', 'AG', 'FSLR', 'FI', 'FSR', 'FIVE', 'FL', 'F', 'FTNT', 'FOXA', 'FCX', 'FUBO', 'FCEL', 'FUTU', 'GME', 'GPS', 'GEHC', 'GNRC', 'GD', 'GE', 'GM', 'GNS', 'GETY', 'GILD', 'GTLB', 'URA', 'GSAT', 'GDDY', 'GS', 'GT', 'GPRO', 'GRWG', 'GSK', 'HAL', 'HBI', 'HOG', 'HE', 'HL', 'HELE', 'HLF', 'HSY', 'HES', 'HD', 'HON', 'HZNP', 'HRL', 'HPQ', 'HSBC', 'HUM', 'HBAN', 'HUT', 'IEP', 'ILMN', 'IMPP', 'INO', 'INTC', 'IBM', 'IP', 'INTU', 'ISRG', 'UUP', 'TAN', 'BKLN', 'IVR', 'QQQ', 'IONQ', 'IQ', 'EWZ', 'EWC', 'EWI', 'SLV', 'TLT', 'IEF', 'FXI', 'IVV', 'ICLN', 'HYG', 'LQD', 'IBB', 'EMB', 'EFA', 'EEM', 'EWU', 'IWM', 'IYR', 'JD', 'JBLU', 'JKS', 'JNJ', 'JPM', 'JMIA', 'JNPR', 'KBH', 'BEKE', 'KVUE', 'KEY', 'KMB', 'KMI', 'KGC', 'KKR', 'KLAC', 'KSS', 'KHC', 'KWEB', 'KR', 'LRCX', 'LVS', 'LMND', 'LEN', 'LII', 'LI', 'LAC', 'LMT', 'LOW', 'LCID', 'LULU', 'LUMN', 'LAZR', 'LYFT', 'M', 'MANU', 'MNKD', 'MARA', 'MRO', 'MPC', 'MAR', 'MRVL', 'MA', 'MTCH', 'MAT', 'MCD', 'MCK', 'MPW', 'MDT', 'MLCO', 'MELI', 'MRK', 'MMAT', 'META', 'MET', 'MCB', 'MGM', 'MU', 'MSFT', 'MSTR', 'MVIS', 'MRNA', 'MDLZ', 'MDB', 'MS', 'MOS', 'MXEF', 'MXEA', 'MULN', 'MRUT', 'XSP', 'NNDM', 'NNOX', 'NANOS', 'NEOG', 'NTAP', 'NFLX', 'NVRO', 'EDU', 'NEGG', 'NEM', 'NEE', 'NKE', 'NKLA', 'NIO', 'NOK', 'JWN', 'NSC', 'NOC', 'NCLH', 'NVAX', 'NVO', 'NU', 'NUE', 'NTR', 'NVDA', 'NXPI', 'ZZK', 'OXY', 'OKTA', 'OLN', 'ONON', 'ON', 'OPEN', 'ORCL', 'OSTK', 'PACW', 'PLTR', 'PANW', 'PARA', 'PYPL', 'PDD', 'BTU', 'PTON', 'PENN', 'PEP', 'PBR', 'PFE', 'PCG', 'PM', 'PSX', 'PHUN', 'PINS', 'PXD', 'PAA', 'PLUG', 'PNC', 'PSNY', 'PPG', 'TROW', 'PG', 'BITO', 'KOLD', 'UCO', 'BOIL', 'SSO', 'TBT', 'TQQQ', 'SPXU', 'SQQQ', 'SDOW', 'SDS', 'UPRO', 'PHM', 'SVXY', 'UVXY', 'VIXY', 'QCOM', 'QS', 'RRC', 'RETA', 'RDFN', 'RDW', 'REGN', 'RH', 'RNG', 'RIOT', 'RIVN', 'HOOD', 'RBLX', 'RKT', 'RKLB', 'ROKU', 'ROST', 'RCL', 'RTX', 'RUM', 'RUT', 'OEX', 'XEO', 'SPGI', 'CRM', 'SRPT', 'SLB', 'SCHW', 'STNG', 'SE', 'STX', 'XLC', 'XLE', 'XLF', 'XLI', 'XLY', 'XLP', 'XLV', 'XLU', 'XLB', 'XLK', 'S', 'NOW', 'SHAK', 'SHEL', 'SHOP', 'SIG', 'SIRI', 'SKLZ', 'SWKS', 'SDC', 'SNAP', 'SNDL', 'SNOW', 'SOFI', 'SEDG', 'SONY', 'SOUN', 'SO', 'LUV', 'SWN', 'DIA', 'GLD', 'FEZ', 'SPY', 'XBI', 'XHB', 'XOP', 'KRE', 'XRT', 'SPLK', 'SPOT', 'SPX', 'SBLK', 'SBUX', 'STEM', 'SFIX', 'STNE', 'SMMT', 'SU', 'SPWR', 'RUN', 'SMCI', 'SNV', 'TMUS', 'TSM', 'TTWO', 'TAL', 'TPR', 'TGT', 'TECK', 'TDOC', 'TELL', 'TSLA', 'WEAT', 'TEVA', 'TXN', 'TGTX', 'CI', 'TTD', 'TMO', 'TLRY', 'TJX', 'TOST', 'TOL', 'TSCO', 'RIG', 'TRIP', 'TFC', 'TUP', 'TWLO', 'TSN', 'UBER', 'PATH', 'ULTA', 'UAA', 'UNP', 'UAL', 'UPS', 'URI', 'X', 'UNG', 'USO', 'UNH', 'U', 'UPST', 'URBN', 'USB', 'UWMC', 'VFC', 'VALE', 'VLO', 'GDX', 'GDXJ', 'OIH', 'SMH', 'VZ', 'VRTX', 'EVTL', 'VERU', 'VKTX', 'SPCE', 'V', 'VOD', 'VRM', 'WBA', 'WMT', 'WBD', 'WM', 'W', 'WB', 'WFC', 'WAL', 'WDC', 'WE', 'WPM', 'WHR', 'WMB', 'WSM', 'WOLF', 'WDAY', 'WKHS', 'WW', 'WYNN', 'XP', 'XPEV', 'YETI', 'YPF', 'Z', 'ZIM', 'ZION', 'ZM', 'ZS', 'VXX', 'INDA', 'ITB']

    # Remove duplicates
    unique_etfs = list(set(etfs))
    return sorted(unique_etfs)

In [3]:
def black_scholes_greeks(S, X, T, r, sigma, option_type='put'):
    d1 = (np.log(S / X) + (r + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    N_d1 = norm.cdf(d1)
    N_d2 = norm.cdf(d2)
    
    N_d1_prime = norm.pdf(d1)
    
    if option_type == 'call':
        C = S * N_d1 - X * np.exp(-r * T) * N_d2
        Delta = N_d1
        Gamma = N_d1_prime / (S * sigma * np.sqrt(T))
        Theta = -(S * N_d1_prime * sigma / (2 * np.sqrt(T))) - r * X * np.exp(-r * T) * N_d2
        Vega = S * np.sqrt(T) * N_d1_prime
        Rho = X * T * np.exp(-r * T) * N_d2
    elif option_type == 'put':
        C = X * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        Delta = N_d1 - 1
        Gamma = N_d1_prime / (S * sigma * np.sqrt(T))
        Theta = -(S * N_d1_prime * sigma / (2 * np.sqrt(T))) + r * X * np.exp(-r * T) * norm.cdf(-d2)
        Vega = S * np.sqrt(T) * N_d1_prime
        Rho = -X * T * np.exp(-r * T) * norm.cdf(-d2)
    else:
        raise ValueError("Invalid option type. Use either 'call' or 'put'.")
        
    return [C, Delta, Gamma, Theta, Vega, Rho]

In [4]:
def fetch_data(ticker, expiration_date):
    try:
        opts = yf.Ticker(ticker).option_chain(expiration_date)
        puts = opts.puts

        if 'bid' not in puts.columns:
            print(f'No options data for {ticker} on {expiration_date}')
            return pd.DataFrame()

        stock_price = round(yf.Ticker(ticker).info['currentPrice'], 2)
        condition = (puts['strike'] < stock_price) & (puts['bid'] > 0)
        puts = puts[condition].copy()
        stock_info = yf.Ticker(ticker).info

        # Add the ETF ticker to the DataFrame
        puts['ETF'] = ticker

        # Add the Stock Price to the DataFrame
        puts['Stock Price'] = stock_price
        # Add how out of the money the put is
        puts['OTM (%)'] = (((puts['Stock Price'] - puts['strike']) / puts['Stock Price']) * 100).round(2)
        # Add the expiration date to the DataFrame
        puts['Expiration Date'] = expiration_date
        # Add 'ROI (%)' column to DataFrame
        puts['ROI (%)'] = ((puts['bid'] / (puts['strike'] - puts['bid'])) * 100).round(2)
        # Add 'highPrice' column to DataFrame
        puts['highPrice'] = puts['bid']
        # Add 'Original ROI (%)' column to DataFrame
        puts['Original ROI (%)'] = ((puts['bid'] / (puts['strike'] - puts['bid'])) * 100).round(2)
        # Add 'Original OTM (%)' column to DataFrame
        puts['Original OTM (%)'] = (((puts['Stock Price'] - puts['strike']) / puts['Stock Price']) * 100).round(2)
        # Add 'Implied Volatility' column to DataFrame
        puts['Implied Volatility'] = puts['impliedVolatility'].round(2)
        # Add 'Original Implied Volatility' column to DataFrame
        puts['Original Implied Volatility'] = puts['impliedVolatility'].round(2)
        # Add 'Original Stock Price' column to DataFrame
        puts['Original Stock Price'] = stock_price
        # Add 'targetHighPrice' column to DataFrame
        puts['targetHighPrice'] = stock_info['targetHighPrice']
        # Add targetLowPrice' column to DataFrame
        puts['targetLowPrice'] = stock_info['targetLowPrice']
        # Add 'targetMeanPrice' column to DataFrame
        puts['targetMeanPrice'] = stock_info['targetMeanPrice']
        # Add 'targetMedianPrice' column to DataFrame
        puts['targetMedianPrice'] = stock_info['targetMedianPrice']
        # Add 'recommendationMean' column to DataFrame
        puts['recommendationMean'] = stock_info['recommendationMean']
        # Add 'recommendationKey' column to DataFrame
        puts['recommendationKey'] = stock_info['recommendationKey']

        # Calculate the number of days until expiration
        today = datetime.now()
        puts['daysToExpiration'] = (pd.to_datetime(puts['Expiration Date']) - today).dt.days

        # Calculate the Greeks
        greeks = black_scholes_greeks(puts['Stock Price'], puts['strike'], puts['daysToExpiration'] / 365, 0.01, puts['impliedVolatility'], option_type='put')
        # Add 'Delta' column to DataFrame
        puts['Delta'] = greeks[1].round(2)
        # Add 'Gamma' column to DataFrame
        puts['Gamma'] = greeks[2].round(2)
        # Add 'Theta' column to DataFrame
        puts['Theta'] = greeks[3].round(2)
        # Add 'Vega' column to DataFrame
        puts['Vega'] = greeks[4].round(2)
        # Add 'Rho' column to DataFrame
        puts['Rho'] = greeks[5].round(2)

        # Fetch historical data for the ETF ticker
        historical_data = yf.Ticker(ticker).history(period="1y")

        # Calculate current moving averages
        current_50_day_MA = historical_data['Close'].rolling(window=50).mean().iloc[-1]
        current_100_day_MA = historical_data['Close'].rolling(window=100).mean().iloc[-1]
        current_200_day_MA = historical_data['Close'].rolling(window=200).mean().iloc[-1]

        # Calculate current RSI
        delta = historical_data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean().iloc[-1]
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean().iloc[-1]
        current_RSI = 100 - (100 / (1 + (gain / loss)))

        # Calculate current MACD
        short_ema = historical_data['Close'].ewm(span=12, adjust=False).mean().iloc[-1]
        long_ema = historical_data['Close'].ewm(span=26, adjust=False).mean().iloc[-1]
        current_MACD = short_ema - long_ema

        # Calculate current VWAP
        typical_price = (historical_data['High'] + historical_data['Low'] + historical_data['Close']) / 3
        current_VWAP = np.cumsum(typical_price * historical_data['Volume']).iloc[-1] / np.cumsum(historical_data['Volume']).iloc[-1]

        # Add single value indicators to puts DataFrame
        puts['50-day MA'] = current_50_day_MA.round(2)
        puts['100-day MA'] = current_100_day_MA.round(2)
        puts['200-day MA'] = current_200_day_MA.round(2)
        puts['RSI'] = current_RSI.round(2)
        puts['MACD'] = current_MACD.round(2)
        puts['VWAP'] = current_VWAP.round(2)
        
        # Calculate 52-Week High and 52-Week Low
        high_52_week = historical_data['Close'].max()
        low_52_week = historical_data['Close'].min()

        puts['52WeekHigh'] = high_52_week.round(2)
        puts['52WeekLow'] = low_52_week.round(2)

        return puts
    except Exception as e:
        print(f"An error occurred: {e}")
        return

In [5]:
def screener(expiration_date):
    # Fetch all ETF tickers from the scraping function
    tickers = manual_etf_tickers()

    results = []

    for ticker in tickers:
        puts = fetch_data(ticker, expiration_date)
        if puts is not None:
            results.append(puts)

    # Check if results is empty
    if results is None:
        return pd.DataFrame()  # Return an empty dataframe

    if len(results) == 0:
        return pd.DataFrame()
    
    all_puts = pd.concat(results)

    return all_puts

In [6]:
def get_batch_ticker_prices(tickers, date):
    # Define Eastern Time Zone
    eastern = pytz.timezone('US/Eastern')

    # Convert date to 'YYYY-MM-DD' format string if it's a datetime.date object
    if isinstance(date, dt.date):  # Using the correct dt.date here
        date = date.strftime('%Y-%m-%d')

    # Convert target_date to Eastern Time Zone
    target_date = eastern.localize(datetime.strptime(date, '%Y-%m-%d'))  # Using datetime class here

    if not is_market_closed(target_date):
        print("The market has not closed yet for this date.")
        return None
    
    # Convert the end date to the next day
    end_date = (target_date + timedelta(days=1)).strftime('%Y-%m-%d')

    try:
        # Fetch data
        data = yf.download(tickers, start=date, end=end_date)
    except Exception as e:
        print(f"An error occurred while fetching data: {e}")
        return None
    
    if data.empty:
        print("No data found.")
        return None

    return data['Close']


In [7]:
def is_market_closed(target_date):
    market_close_time = dt.time(16, 0)
    
    # Define Eastern Time Zone
    eastern = pytz.timezone('US/Eastern')
    
    # Check if target_date is already timezone-aware
    if target_date.tzinfo is None or target_date.tzinfo.utcoffset(target_date) is None:
        # Convert target_date to Eastern Time Zone
        target_date = eastern.localize(target_date)
    else:
        # Convert to Eastern Time if it's in a different timezone
        target_date = target_date.astimezone(eastern)
    
    # Get the current time in Eastern Time Zone
    current_time = datetime.now(eastern).time()

    # Check if the market has closed for the target date
    if target_date.date() < datetime.now(eastern).date() or \
       (target_date.date() == datetime.now(eastern).date() and current_time >= market_close_time):
        return True
    else:
        return False

In [8]:
def next_four_fridays():
    # Get today's date and time in EST timezone
    est = pytz.timezone('US/Eastern')
    today = datetime.now(est)
    
    # Find the next Friday
    days_until_friday = (4 - today.weekday() + 7) % 7
    next_friday = today + timedelta(days=days_until_friday)
    
    # Reset the time to 4:30 PM on the next Friday
    next_friday = next_friday.replace(hour=16, minute=30, second=0, microsecond=0)
    
    # If today is Friday and it's past 4:30 PM, consider the next week's Friday
    if today.weekday() == 4 and today.time() > next_friday.time():
        next_friday += timedelta(days=7)
    
    # Remove time from datetime for output
    next_friday = next_friday.date()
    
    # Find the Friday after the next
    following_friday = next_friday + timedelta(days=7)
    
    # Find the Friday after the following
    next_next_friday = following_friday + timedelta(days=7)
    
    # Find the Friday after the next next
    next_next_next_friday = next_next_friday + timedelta(days=7)
    
    return next_friday.strftime('%Y-%m-%d'), following_friday.strftime('%Y-%m-%d'), next_next_friday.strftime('%Y-%m-%d'), next_next_next_friday.strftime('%Y-%m-%d')

# Get the next four Fridays
friday1, friday2, friday3, friday4 = next_four_fridays()
print(f"The next four Fridays are: {friday1}, {friday2}, {friday3}, and {friday4}")

The next four Fridays are: 2023-10-06, 2023-10-13, 2023-10-20, and 2023-10-27


### Sorting the data

In [9]:
file_name = 'Data/naked_puts_results.csv'

# File Reading with Error Handling
try:
    df = pd.read_csv(file_name)
except FileNotFoundError:
    print(f"Error: {file_name} not found.")
    exit(1)


In [10]:
# Get the current date and time
today_datetime = datetime.now(pytz.timezone('US/Eastern'))

# Convert the date to YYYY-MM-DD format
todayStr = today_datetime.strftime("%Y-%m-%d")

# Convert todayStr to a datetime date object for more accurate comparison
today_date = pd.to_datetime(todayStr).date()

# Get the current time in HH:MM format
current_time = today_datetime.strftime("%H:%M")

# Print the date and time
print("Today's date:", todayStr)
print("Current Eastern Time:", current_time)

# Convert date strings to datetime objects for more reliable comparison
expiration_dates = pd.to_datetime(df['Expiration Date']).dt.date.unique()

# Create an empty list to store the expired dates
expiredOptionDates = []

# Check if today's date is greater than any of the dates in the expiration_dates
for date in expiration_dates:
    if today_date > date or (today_date == date and current_time >= '16:30'):
        expiredOptionDates.append(date)

if expiredOptionDates:
    for date in expiredOptionDates:
        date_str = date.strftime('%Y-%m-%d')
        currentDf = df[df['Expiration Date'] == date_str].copy()  # Make a copy if you intend to modify this slice

        if currentDf.empty:
            print(f"No data found for date {date_str}")
            continue

        unique_etfs = list(currentDf['ETF'].unique())
        tickerPrices = get_batch_ticker_prices(unique_etfs, date)

        if tickerPrices is None:
            print("Ticker prices could not be fetched.")
            continue

        # Convert strike column to float for numerical comparison
        currentDf.loc[:, 'strike'] = currentDf['strike'].astype(float)

        # Go through each row in currentDf
        for index, row in currentDf.iterrows():
            etf_price = tickerPrices.get(row['ETF'], None)
            
            if etf_price is None:
                print(f"No price information found for ETF {row['ETF']}.")
                continue

            if isinstance(etf_price, pd.Series):
                etf_price = etf_price.iloc[0]

            currentDf.loc[index, 'Stock Price'] = etf_price

            if row['strike'] < etf_price:
                currentDf.loc[index, 'Status'] = 'Success'
            else:
                currentDf.loc[index, 'Status'] = 'Failed'

        csv_file_path = 'Data/putsDataSuccessFailed.csv'
         # Check if the naked_puts_results.csv file exists and if it's not empty
        if os.path.exists(csv_file_path) and os.path.getsize(csv_file_path) > 0:
            
            # Read the naked_puts_results.csv file into a DataFrame
            file_df = pd.read_csv(csv_file_path)
            
            # Find common contractSymbols
            common_symbols = set(file_df['contractSymbol']).intersection(set(currentDf['contractSymbol']))

            for symbol in common_symbols:
                # Replace all column values for rows with this contractSymbol
                currentDf.loc[currentDf['contractSymbol'] == symbol] = file_df.loc[file_df['contractSymbol'] == symbol].values
                
            # Append new rows
            desired_columns = ['contractSymbol','Expiration Date', 'ETF', 'Stock Price', 'lastPrice', 'bid', 'ask', 'strike', 'ROI (%)', 'OTM (%)', 'volume', 'openInterest', 'Implied Volatility', 'highPrice', 'Original Stock Price','Original ROI (%)', 
            'Original OTM (%)', 'Original Implied Volatility', '50-day MA', '100-day MA', '200-day MA', 'RSI', 'MACD', 'VWAP', 'Delta', 'Gamma', 'Theta', 'Vega', 'Rho', '52WeekHigh', '52WeekLow', 'targetHighPrice', 'targetLowPrice', 'targetMeanPrice',
            'targetMedianPrice', 'recommendationMean', 'recommendationKey', 'Status']
            new_rows = file_df.loc[~file_df['contractSymbol'].isin(currentDf['contractSymbol']), desired_columns]
            currentDf = pd.concat([currentDf, new_rows], ignore_index=False)
            
        # Save the updated results DataFrame back to the naked_puts_results.csv file
        currentDf.to_csv(csv_file_path, index=False)

# Change the dates in expiredOptionDates to strings using .strftime('%Y-%m-%d')
expiredOptionDates = [date.strftime('%Y-%m-%d') for date in expiredOptionDates]

# Remove all the expiredOptionDates in df
df = df[~df['Expiration Date'].isin(expiredOptionDates)]

# Save the updated results DataFrame back to the naked_puts_results.csv file
df.to_csv(file_name, index=False)

Today's date: 2023-09-29
Current Eastern Time: 17:15
[*********************100%%**********************]  66 of 66 completed


### Select the fridays you want from the next four fridays

In [11]:
fridays = [friday1]

### Looking into through Fridays ahead

In [12]:
# Check if the CSV file exists
if os.path.exists('Data/naked_puts_results.csv'):
    try:
        # Read the CSV file into a DataFrame
        df_check = pd.read_csv('Data/naked_puts_results.csv')
        
        # Check if 'friday1' exists in the 'Expiration Date' column
        if friday1 in df_check['Expiration Date'].values:
            print(f"Data for {friday1} already exists in the DataFrame.")
        else:
            # List of desired columns
            desired_columns = ['contractSymbol', 'Expiration Date', 'ETF', 'Stock Price', 'lastPrice', 'bid', 'ask', 'strike', 'ROI (%)', 'OTM (%)', 'volume', 'openInterest', 'Implied Volatility', 'highPrice', 'Original Stock Price', 'Original ROI (%)', 'Original OTM (%)', 'Original Implied Volatility', '50-day MA', '100-day MA', '200-day MA', 'RSI', 'MACD', 'VWAP', 'Delta', 'Gamma', 'Theta', 'Vega', 'Rho', '52WeekHigh', '52WeekLow', 'targetHighPrice', 'targetLowPrice', 'targetMeanPrice', 'targetMedianPrice', 'recommendationMean', 'recommendationKey']

            # Initialize results as an empty DataFrame with columns
            results = pd.DataFrame(columns=desired_columns)

            # Loop through each Friday
            for x in fridays:
                # Get the screener DataFrame for each Friday
                screener_df = screener(x)
                
                # Check if the DataFrame is empty
                if screener_df.empty:
                    print(f"No data found for {x}")
                    continue
                
                # Append the screener DataFrame to results
                results = pd.concat([results, screener_df], ignore_index=True)

            # List of columns to display
            results = results[desired_columns]

            # Make results a DataFrame
            results = pd.DataFrame(results)
            
    except pd.errors.EmptyDataError:
        # Handle the EmptyDataError by skipping the read operation
        print("CSV file is empty or has no columns to parse.")
        # List of desired columns
        desired_columns = ['contractSymbol', 'Expiration Date', 'ETF', 'Stock Price', 'lastPrice', 'bid', 'ask', 'strike', 'ROI (%)', 'OTM (%)', 'volume', 'openInterest', 'Implied Volatility', 'highPrice', 'Original Stock Price', 'Original ROI (%)', 'Original OTM (%)', 'Original Implied Volatility', '50-day MA', '100-day MA', '200-day MA', 'RSI', 'MACD', 'VWAP', 'Delta', 'Gamma', 'Theta', 'Vega', 'Rho', '52WeekHigh', '52WeekLow', 'targetHighPrice', 'targetLowPrice', 'targetMeanPrice', 'targetMedianPrice', 'recommendationMean', 'recommendationKey']

        # Initialize results as an empty DataFrame with columns
        results = pd.DataFrame(columns=desired_columns)

        # Loop through each Friday
        for x in fridays:
            # Get the screener DataFrame for each Friday
            screener_df = screener(x)
            
            # Check if the DataFrame is empty
            if screener_df.empty:
                print(f"No data found for {x}")
                continue
            
            # Append the screener DataFrame to results
            results = pd.concat([results, screener_df], ignore_index=True)

        # List of columns to display
        results = results[desired_columns]

        # Make results a DataFrame
        results = pd.DataFrame(results)
else:
    print("CSV file 'naked_puts_results.csv' does not exist.")

An error occurred: Expiration `2023-10-06` cannot be found. Available expirations are: [2023-10-20, 2024-01-19, 2024-04-19]
An error occurred: Expiration `2023-10-06` cannot be found. Available expirations are: [2023-10-20, 2023-12-15, 2024-03-15]
An error occurred: Expiration `2023-10-06` cannot be found. Available expirations are: [2023-10-20, 2023-11-17, 2024-01-19, 2024-04-19, 2025-01-17]
An error occurred: Expiration `2023-10-06` cannot be found. Available expirations are: [2023-10-20, 2023-11-17, 2023-12-15, 2024-03-15]
An error occurred: Expiration `2023-10-06` cannot be found. Available expirations are: [2023-10-20, 2023-11-17, 2024-01-19, 2024-04-19, 2025-01-17, 2026-01-16]
An error occurred: Expiration `2023-10-06` cannot be found. Available expirations are: [2023-10-20, 2023-11-17, 2024-01-19, 2024-04-19]
An error occurred: Expiration `2023-10-06` cannot be found. Available expirations are: [2023-10-20, 2023-11-17, 2024-02-16, 2024-05-17]
An error occurred: Expiration `2023-

### Saving the data into CSV files

In [13]:
file_path = 'Data/naked_puts_results.csv'
results.to_csv(file_path, index=False)